# **Image preprocessing, instance segmentation (omnipose), object cropping master**
***This notebook creates RGB PNGs from 4 Grayscale Tif imgs, normalizes the intensity values of each channel (by plate), and outputs padded single cell (or object) PNGs based on a greyscale masks produced in Cellpose.***

---
***Prerequisites:

    1. Starting grayscale imgs should be in a folder with the plateID metadata in the foldername (last foldername)
    2. Image names must all be the same length e.g. 'A01_T0001F001L01A01Z01C03.tif'
    3. Image names must contain sufficient metadata i.e., wellID, fieldID, channelID
    3. No '_' or whitespaces in any foldernames***

---   
****To run localy:

    1. Install Anaconda
    2. open an the anaconda comand prompt
    3. create a jupyter environment named jupyter by running the following code:
        conda create -n jupyter python==3.6
    4. activate the environment: 
        activate jupyter
    5. install jupyter notebooks in the jupyter environment
        conda install jupyter
    6. create another environemnt that will be used as the kernel
        conda create -n jupyter_env python==3.6
    7. activate the environment: 
        1. activate jupyter_env
        2. install ipykernal and conect the env to localhost: 
        conda install ipykernel
        python -m ipykernel install --user --name=jupyter_env
    8. To start a usin jupyter:
        activate jupyter
        jupyter notebook
    9. To install the notebooks bependencies:
       activate jupyter_env
       conda install opencv numpy pandas pillow skimage***

---



In [ ]:
from jupyterthemes import jtplot
# choose which theme to inherit plotting style from
# onedork | grade3 | oceans16 | chesterish | monokai | solarizedl | solarizedd
jtplot.style(theme='onedork')

import torch
import random
import shutil
import numpy as np
import os
import sys
import shutil 
import os
import random
import pandas as pd
from glob import glob
import cv2
import csv
from skimage import io
import ipywidgets as widgets
from datetime import datetime
from skimage import img_as_float32
from tifffile import imread, imsave
from skimage.util import img_as_uint
from skimage.util import img_as_ubyte
from urllib.parse import urlparse
from cellpose import models
from ipywidgets import interact, interact_manual
from zipfile import ZIP_DEFLATED
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
from cellpose import plot
from skimage import exposure
from os import environ

Image.MAX_IMAGE_PIXELS = None
%load_ext memory_profiler
%matplotlib inline

!nvidia-smi
use_GPU=True

print('Torch available: ', torch.cuda.is_available())
print(torch.version.cuda)

#before = [str(m) for m in sys.modules]
#use_GPU = models.use_gpu()
device = torch.device('cuda:0')

In [ ]:
#This function mooves images based on channel to channel folders 
#and extracts image metadata from img filename and renames imgs.
def move_to_chan_folder(src, nr_of_channels):
    if nr_of_channels == 1:
        if os.path.exists(src+'/'+'chan_1') is False:         
            os.mkdir(src+'/chan_1')
    if nr_of_channels == 2:
        if os.path.exists(src+'/'+'chan_1') is False:         
            os.mkdir(src+'/chan_1')
        if os.path.exists(src+'/'+'chan_2') is False:         
            os.mkdir(src+'/chan_2')
    if nr_of_channels == 3:
        if os.path.exists(src+'/'+'chan_1') is False:         
            os.mkdir(src+'/chan_1')
        if os.path.exists(src+'/'+'chan_2') is False:         
            os.mkdir(src+'/chan_2')
        if os.path.exists(src+'/'+'chan_3') is False:         
            os.mkdir(src+'/chan_3')
    if nr_of_channels == 4:
        if os.path.exists(src+'/'+'chan_1') is False:         
            os.mkdir(src+'/chan_1')
        if os.path.exists(src+'/'+'chan_2') is False:         
            os.mkdir(src+'/chan_2')
        if os.path.exists(src+'/'+'chan_3') is False:         
            os.mkdir(src+'/chan_3')
        if os.path.exists(src+'/'+'chan_4') is False:         
            os.mkdir(src+'/chan_4')
    if nr_of_channels == 5:
        if os.path.exists(src+'/'+'chan_1') is False:         
            os.mkdir(src+'/chan_1')
        if os.path.exists(src+'/'+'chan_2') is False:         
            os.mkdir(src+'/chan_2')
        if os.path.exists(src+'/'+'chan_3') is False:         
            os.mkdir(src+'/chan_3')
        if os.path.exists(src+'/'+'chan_4') is False:         
            os.mkdir(src+'/chan_4')
        if os.path.exists(src+'/'+'chan_5') is False:         
            os.mkdir(src+'/chan_5')
    if nr_of_channels == 6:
        if os.path.exists(src+'/'+'chan_1') is False:         
            os.mkdir(src+'/chan_1')
        if os.path.exists(src+'/'+'chan_2') is False:         
            os.mkdir(src+'/chan_2')
        if os.path.exists(src+'/'+'chan_3') is False:         
            os.mkdir(src+'/chan_3')
        if os.path.exists(src+'/'+'chan_4') is False:         
            os.mkdir(src+'/chan_4')
        if os.path.exists(src+'/'+'chan_5') is False:         
            os.mkdir(src+'/chan_5')
        if os.path.exists(src+'/'+'chan_6') is False:         
            os.mkdir(src+'/chan_6')
    if nr_of_channels == 7:
        if os.path.exists(src+'/'+'chan_1') is False:         
            os.mkdir(src+'/chan_1')
        if os.path.exists(src+'/'+'chan_2') is False:         
            os.mkdir(src+'/chan_2')
        if os.path.exists(src+'/'+'chan_3') is False:         
            os.mkdir(src+'/chan_3')
        if os.path.exists(src+'/'+'chan_4') is False:         
            os.mkdir(src+'/chan_4')
        if os.path.exists(src+'/'+'chan_5') is False:         
            os.mkdir(src+'/chan_5')
        if os.path.exists(src+'/'+'chan_6') is False:         
            os.mkdir(src+'/chan_6')
        if os.path.exists(src+'/'+'chan_7') is False:         
            os.mkdir(src+'/chan_7')
    if nr_of_channels == 8:
        if os.path.exists(src+'/'+'chan_1') is False:         
            os.mkdir(src+'/chan_1')
        if os.path.exists(src+'/'+'chan_2') is False:         
            os.mkdir(src+'/chan_2')
        if os.path.exists(src+'/'+'chan_3') is False:         
            os.mkdir(src+'/chan_3')
        if os.path.exists(src+'/'+'chan_4') is False:         
            os.mkdir(src+'/chan_4')
        if os.path.exists(src+'/'+'chan_5') is False:         
            os.mkdir(src+'/chan_5')
        if os.path.exists(src+'/'+'chan_6') is False:         
            os.mkdir(src+'/chan_6')
        if os.path.exists(src+'/'+'chan_7') is False:         
            os.mkdir(src+'/chan_7')
        if os.path.exists(src+'/'+'chan_8') is False:         
            os.mkdir(src+'/chan_8')
    if nr_of_channels == 9:                     
        if os.path.exists(src+'/'+'chan_1') is False:         
            os.mkdir(src+'/chan_1')
        if os.path.exists(src+'/'+'chan_2') is False:         
            os.mkdir(src+'/chan_2')
        if os.path.exists(src+'/'+'chan_3') is False:         
            os.mkdir(src+'/chan_3')
        if os.path.exists(src+'/'+'chan_4') is False:         
            os.mkdir(src+'/chan_4')
        if os.path.exists(src+'/'+'chan_5') is False:         
            os.mkdir(src+'/chan_5')
        if os.path.exists(src+'/'+'chan_6') is False:         
            os.mkdir(src+'/chan_6')
        if os.path.exists(src+'/'+'chan_7') is False:         
            os.mkdir(src+'/chan_7')
        if os.path.exists(src+'/'+'chan_8') is False:         
            os.mkdir(src+'/chan_8')
        if os.path.exists(src+'/'+'chan_9') is False:         
            os.mkdir(src+'/chan_9')
    for root, _, files in os.walk(src):
        for file in files:
            path = os.path.join(src, file)
            if os.path.exists(path) == False:
                print(file, ': is in subfolder')
            else:
                print(file, ': is in ', src)
                if os.path.exists(src+'/'+file) == True:
                    metadata=os.path.splitext(file)[0]
                    extension=os.path.splitext(file)[1]
                    if extension == '.tif':
                        img_path=os.path.join(src, file)
                        plateid = os.path.basename(src)
                        wellid=metadata[0:3]
                        fieldid=metadata[10:13]
                        chanid=metadata[22:25]
                        newname=plateid+'_'+wellid+'_'+fieldid
                        new_path=os.path.join(src, newname+extension)
                        print(metadata, plateid, wellid, fieldid, chanid)
                        if chanid == 'C01' or 'C1':
                            move=src+'/'+'chan_1'
                        if chanid == 'C02' or 'C2':
                            move=src+'/'+'chan_2'
                        if chanid == 'C03' or 'C3':
                            move=src+'/'+'chan_3'
                        if chanid == 'C04' or 'C4':
                            move=src+'/'+'chan_4'
                        if chanid == 'C05' or 'C5':
                            move=src+'/'+'chan_5'
                        if chanid == 'C06' or 'C6':
                            move=src+'/'+'chan_6'
                        if chanid == 'C07' or 'C7':
                            move=src+'/'+'chan_7'
                        if chanid == 'C08' or 'C8':
                            move=src+'/'+'chan_8'
                        if chanid == 'C09' or 'C9':
                            move=src+'/'+'chan_9'
                        if chanid != 'C01' or 'C02' or 'C03' or 'C04' or 'C05' or 'C06' or 'C07' or 'C08' or 'C09':
                            print('chanid metadata not found')
                            pass
                        new_path=os.path.join(move, newname+extension)
                        shutil.move(img_path, new_path)

# in case you want something like mean over this H slices:
#Function that colects 4 or 3 grayscale images from a folder and outputs renamed 16 bit PNGs.
#this function also colects low and high quantile intensity values for each chanel, which are used in the next function.
def gray_to_color(src, nr_of_channels, mode):
    if mode == 'gray':
        if os.path.exists(src+'/'+'gray') is False:
            os.mkdir(src+'/'+'gray')
        if os.path.exists(src+'/'+'gray/img_intensity.csv') is False:         
            with open(src+'/'+'gray/img_intensity.csv', 'w') as csvfile:
                img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99'])
        for root, _, files in os.walk(src+'/chan_1'):
            for file in files:
                gray_path=os.path.join(src+'/'+'gray', file)
                if os.path.exists(gray_path) == True:
                    print('Gray '+file+' exists')
                    pass
                else:
                    gray = cv2.imread(chan_1_path, -1)
                    
                    chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                    chan_1_ratio_1 = int(chan_1_int_Q99/chan_1_int_Q01)
                    if chan_1_ratio_1 < 10:
                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.999))
                        chan_1_ratio_2 = int(chan_1_int_Q99/chan_1_int_Q01)
                        if chan_1_ratio_2 >= 10:
                            print('WARNING: Q99 set to 0.999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2)
                        if chan_1_ratio_2 < 10:
                            chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.001, 0.9999))
                            chan_1_ratio_3 = int(chan_1_int_Q99/chan_1_int_Q01)
                            print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2, chan_1_ratio_3)

                    print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99')
                    print([file, chan_1_int_Q01, chan_1_int_Q99])
                    new_row = [gray_path, chan_1_int_Q01, chan_1_int_Q99]
                    img_intensity_csv = src+'/gray/img_intensity.csv'
                    with open(img_intensity_csv, 'a+', newline='') as write_obj:
                        csv_writer = csv.writer(write_obj)
                        csv_writer.writerow(new_row)
                    print('Gray dtype: '+str(gray.dtype), 'Gray dimensions : '+str(gray.shape))
                    cv2.imwrite(gray_path, gray.astype(bitdepth))

    if mode == 'rgb':
        if os.path.exists(src+'/'+'rgb') is False:
            os.mkdir(src+'/'+'rgb')
        if nr_of_channels == 2:
            if os.path.exists(src+'/'+'rgb/img_intensity.csv') is False:         
                with open(src+'/'+'rgb/img_intensity.csv', 'w') as csvfile:
                    img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99'])
            for root, _, files in os.walk(src+'/chan_1'):
                for file in files:
                    chan_1_path=os.path.join(src+'/chan_1', file)
                    chan_2_path=os.path.join(src+'/chan_2', file)
                    rgb_path=os.path.join(src+'/'+'rgb', file)
                    if os.path.exists(rgb_path) == True:
                        print('rgb '+file+' exists')
                        pass
                    else:
                        chan_1 = cv2.imread(chan_1_path, -1)
                        chan_2 = cv2.imread(chan_2_path, -1)

                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                        chan_1_ratio_1 = int(chan_1_int_Q99/chan_1_int_Q01)
                        if chan_1_ratio_1 < 10:
                            chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.999))
                            chan_1_ratio_2 = int(chan_1_int_Q99/chan_1_int_Q01)
                            if chan_1_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2)
                            if chan_1_ratio_2 < 10:
                                chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.001, 0.9999))
                                chan_1_ratio_3 = int(chan_1_int_Q99/chan_1_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2, chan_1_ratio_3)
                        chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.99))
                        chan_2_ratio_1 = int(chan_2_int_Q99/chan_2_int_Q01)
                        if chan_2_ratio_1 < 10:
                            chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.999))
                            chan_2_ratio_2 = int(chan_2_int_Q99/chan_2_int_Q01)
                            if chan_2_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2)
                            if chan_2_ratio_2 < 10:
                                chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.001, 0.9999))
                                chan_2_ratio_3 = int(chan_2_int_Q99/chan_2_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2, chan_2_ratio_3)
                        
                        rgb = cv2.merge((chan_1, chan_2),-1)
                        
                        print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99')
                        print([file, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99])
                        new_row = [rgb_path, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99]
                        img_intensity_csv = src+'/rgb/img_intensity.csv'
                        with open(img_intensity_csv, 'a+', newline='') as write_obj:
                            csv_writer = csv.writer(write_obj)
                            csv_writer.writerow(new_row)
                        print('RGB dtype: '+ str(rgb.dtype), 'RGB dimensions : '+str(rgb.shape))
                        cv2.imwrite(rgb_path, rgb.astype(bitdepth))                    
                    
        if nr_of_channels == 3:
            if os.path.exists(src+'/'+'rgb/img_intensity.csv') is False:         
                with open(src+'/'+'rgb/img_intensity.csv', 'w') as csvfile:
                    img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99'])
            for root, _, files in os.walk(src+'/chan_1'):
                for file in files:
                    chan_1_path=os.path.join(src+'/chan_1', file)
                    chan_2_path=os.path.join(src+'/chan_2', file)
                    chan_3_path=os.path.join(src+'/chan_3', file)
                    chan_4_path=os.path.join(src+'/chan_4', file)
                    rgb_path=os.path.join(src+'/'+'rgb', file)
                    if os.path.exists(rgb_path) == True:
                        print('rgb '+file+' exists')
                        pass
                    else:
                        chan_1 = cv2.imread(chan_1_path, -1)
                        chan_2 = cv2.imread(chan_2_path, -1)
                        chan_3 = cv2.imread(chan_3_path, -1)

                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                        chan_1_ratio_1 = int(chan_1_int_Q99/chan_1_int_Q01)
                        if chan_1_ratio_1 < 10:
                            chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.999))
                            chan_1_ratio_2 = int(chan_1_int_Q99/chan_1_int_Q01)
                            if chan_1_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2)
                            if chan_1_ratio_2 < 10:
                                chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.001, 0.9999))
                                chan_1_ratio_3 = int(chan_1_int_Q99/chan_1_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2, chan_1_ratio_3)
                        chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.99))
                        chan_2_ratio_1 = int(chan_2_int_Q99/chan_2_int_Q01)
                        if chan_2_ratio_1 < 10:
                            chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.999))
                            chan_2_ratio_2 = int(chan_2_int_Q99/chan_2_int_Q01)
                            if chan_2_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2)
                            if chan_2_ratio_2 < 10:
                                chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.001, 0.9999))
                                chan_2_ratio_3 = int(chan_2_int_Q99/chan_2_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2, chan_2_ratio_3)
                        chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.99))
                        chan_3_ratio_1 = int(chan_3_int_Q99/chan_3_int_Q01)
                        if chan_3_ratio_1 < 10:
                            chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.999))
                            chan_3_ratio_2 = int(chan_3_int_Q99/chan_3_int_Q01)
                            if chan_3_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2)
                            if chan_3_ratio_2 < 10:
                                chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.001, 0.9999))
                                chan_3_ratio_3 = int(chan_3_int_Q99/chan_3_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2, chan_3_ratio_3)

                        
                        rgb = cv2.merge((chan_1, chan_2, chan_3),-1)
                        
                        print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99')
                        print([file, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99])
                        new_row = [rgb_path, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99]
                        img_intensity_csv = src+'/rgb/img_intensity.csv'
                        with open(img_intensity_csv, 'a+', newline='') as write_obj:
                            csv_writer = csv.writer(write_obj)
                            csv_writer.writerow(new_row)
                        print('RGB dtype: '+ str(rgb.dtype), 'RGB dimensions : '+str(rgb.shape))
                        cv2.imwrite(rgb_path, rgb.astype(bitdepth))
                        
    if mode == 'rgba': 
        if nr_of_channels == 4:
            if os.path.exists(src+'/'+'rgba') is False:
                os.mkdir(src+'/'+'rgba')
            if os.path.exists(src+'/rgba/img_intensity.csv') is False:         
                with open(src+'/rgba/img_intensity.csv', 'w') as csvfile:
                    img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99'])

            for root, _, files in os.walk(src+'/chan_1'):
                for file in files:
                    chan_1_path=os.path.join(src+'/chan_1', file)
                    chan_2_path=os.path.join(src+'/chan_2', file)
                    chan_3_path=os.path.join(src+'/chan_3', file)
                    chan_4_path=os.path.join(src+'/chan_4', file)
                    rgba_path=os.path.join(src+'/'+'rgba', file)
                    if os.path.exists(rgba_path) == True:
                        print('rgba '+file+' exists')
                        pass
                    else:
                        chan_1 = cv2.imread(chan_1_path, -1)
                        chan_2 = cv2.imread(chan_2_path, -1)
                        chan_3 = cv2.imread(chan_3_path, -1)
                        chan_4 = cv2.imread(chan_4_path, -1)
                        
                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                        chan_1_ratio_1 = int(chan_1_int_Q99/chan_1_int_Q01)
                        if chan_1_ratio_1 < 10:
                            chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.999))
                            chan_1_ratio_2 = int(chan_1_int_Q99/chan_1_int_Q01)
                            if chan_1_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2)
                            if chan_1_ratio_2 < 10:
                                chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.001, 0.9999))
                                chan_1_ratio_3 = int(chan_1_int_Q99/chan_1_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2, chan_1_ratio_3)
                        chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.99))
                        chan_2_ratio_1 = int(chan_2_int_Q99/chan_2_int_Q01)
                        if chan_2_ratio_1 < 10:
                            chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.999))
                            chan_2_ratio_2 = int(chan_2_int_Q99/chan_2_int_Q01)
                            if chan_2_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2)
                            if chan_2_ratio_2 < 10:
                                chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.001, 0.9999))
                                chan_2_ratio_3 = int(chan_2_int_Q99/chan_2_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2, chan_2_ratio_3)
                        chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.99))
                        chan_3_ratio_1 = int(chan_3_int_Q99/chan_3_int_Q01)
                        if chan_3_ratio_1 < 10:
                            chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.999))
                            chan_3_ratio_2 = int(chan_3_int_Q99/chan_3_int_Q01)
                            if chan_3_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2)
                            if chan_3_ratio_2 < 10:
                                chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.001, 0.9999))
                                chan_3_ratio_3 = int(chan_3_int_Q99/chan_3_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2, chan_3_ratio_3)
                        chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.99))
                        chan_4_ratio_1 = int(chan_4_int_Q99/chan_4_int_Q01)
                        if chan_4_ratio_1 < 10:
                            chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.999))
                            chan_4_ratio_2 = int(chan_4_int_Q99/chan_4_int_Q01)
                            if chan_4_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2)
                            if chan_4_ratio_2 <10:
                                chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.001, 0.9999))
                                chan_4_ratio_3 = int(chan_4_int_Q99/chan_4_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2, chan_4_ratio_3)
                        
                        rgba = cv2.merge((chan_1, chan_2, chan_3, chan_4),-1)
                        
                        print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99')
                        print([file, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99])
                        new_row = [rgba_path, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99]
                        img_intensity_csv = src+'/rgba/img_intensity.csv'
                        with open(img_intensity_csv, 'a+', newline='') as write_obj:
                            csv_writer = csv.writer(write_obj)
                            csv_writer.writerow(new_row)
                        print('RGBA dtype: '+ str(rgba.dtype), 'RGBA dimensions : '+str(rgba.shape))
                        cv2.imwrite(rgba_path, rgba.astype(bitdepth))
    
    if mode == 'stack':
        if os.path.exists(src+'/'+'stack') is False:
            os.mkdir(src+'/'+'stack')
        
        if nr_of_channels == 1:
            if os.path.exists(src+'/'+'stack/img_intensity.csv') is False:         
                with open(src+'/'+'stack/img_intensity.csv', 'w') as csvfile:
                    img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99'])
            for root, _, files in os.walk(src+'/chan_1'):
                for file in files:
                    chan_1_path=os.path.join(src+'/'+'chan_1', file)
                    stack_path=os.path.join(src+'/'+'stack', file)
                    if os.path.exists(stack_path) == True:
                        print('stack '+file+' exists')
                        pass     
                    else:
                        chan_1 = cv2.imread(chan_1_path, -1)
                        stack = np.stack([chan_1], axis=-1)
                        
                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                        
                        print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99')
                        print([file, chan_1_int_Q01, chan_1_int_Q99])
                        new_row = [rgb_path, chan_1_int_Q01, chan_1_int_Q99]
                        img_intensity_csv = src+'/stack/img_intensity.csv'
                        with open(img_intensity_csv, 'a+', newline='') as write_obj:
                            csv_writer = csv.writer(write_obj)
                            csv_writer.writerow(new_row)                    
                        print('Stack dtype: '+str(stack.dtype), 'Stack dimensions : '+str(stack.shape))
                        np.save(stack_path, stack.astype(bitdepth))
                                            
        if nr_of_channels == 2:
            if os.path.exists(src+'/'+'stack/img_intensity.csv') is False:         
                with open(src+'/'+'stack/img_intensity.csv', 'w') as csvfile:
                    img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99'])
            for root, _, files in os.walk(src+'/chan_1'):
                for file in files:
                    chan_1_path=os.path.join(src+'/'+'chan_1', file)
                    chan_2_path=os.path.join(src+'/'+'chan_2', file)
                    stack_path=os.path.join(src+'/'+'stack', file)
                    if os.path.exists(stack_path) == True:
                        print('stack '+file+' exists')
                        pass     
                    else:                    
                        chan_1 = cv2.imread(chan_1_path, -1)
                        chan_2 = cv2.imread(chan_2_path, -1)
                        stack = np.stack([chan_1, chan_2], axis=-1)
                        
                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                        chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.99))
                        
                        print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99')
                        print([file, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99])
                        new_row = [stack_path, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99]
                        img_intensity_csv = src+'/stack/img_intensity.csv'
                        with open(img_intensity_csv, 'a+', newline='') as write_obj:
                            csv_writer = csv.writer(write_obj)
                            csv_writer.writerow(new_row)
                        print('Stack dtype: '+str(stack.dtype), 'Stack dimensions : '+str(stack.shape))
                        np.save(stack_path, stack.astype(bitdepth))
                                                    
        if nr_of_channels == 3:
            if os.path.exists(src+'/'+'stack/img_intensity.csv') is False:         
                with open(src+'/'+'stack/img_intensity.csv', 'w') as csvfile:
                    img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99'])
            for root, _, files in os.walk(src+'/chan_1'):
                for file in files:
                    chan_1_path=os.path.join(src+'/'+'chan_1', file)
                    chan_2_path=os.path.join(src+'/'+'chan_2', file)
                    chan_3_path=os.path.join(src+'/'+'chan_3', file)
                    stack_path=os.path.join(src+'/'+'stack', file)
                    if os.path.exists(stack_path) == True:
                        print('stack '+file+' exists')
                        pass     
                    else:                    
                        chan_1 = cv2.imread(chan_1_path, -1)
                        chan_2 = cv2.imread(chan_2_path, -1)
                        chan_3 = cv2.imread(chan_3_path, -1)
                        stack = np.stack([chan_1, chan_2, chan_3], axis=-1)
                        
                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                        chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.99))
                        chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.99))
                        
                        print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99')
                        print([file, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99])
                        new_row = [stack_path, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99]
                        img_intensity_csv = src+'/stack/img_intensity.csv'
                        with open(img_intensity_csv, 'a+', newline='') as write_obj:
                            csv_writer = csv.writer(write_obj)
                            csv_writer.writerow(new_row)
                        print('Stack dtype: '+str(stack.dtype), 'Stack dimensions : '+str(stack.shape))
                        np.save(stack_path, stack.astype(bitdepth))
                                                    
        if nr_of_channels == 4:
            if os.path.exists(src+'/'+'stack/img_intensity.csv') is False:         
                with open(src+'/'+'stack/img_intensity.csv', 'w') as csvfile:
                    img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99'])
                    print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99')
            for root, _, files in os.walk(src+'/chan_1'):
                for file in files:
                    name=os.path.splitext(file)[0]
                    chan_1_path=os.path.join(src+'/'+'chan_1', file)
                    chan_2_path=os.path.join(src+'/'+'chan_2', file)
                    chan_3_path=os.path.join(src+'/'+'chan_3', file)
                    chan_4_path=os.path.join(src+'/'+'chan_4', file)
                    stack_path=os.path.join(src+'/'+'stack', name+'.npy')
                    if os.path.exists(stack_path) == True:
                        print('stack '+file+' exists')
                        pass     
                    else:                    
                        chan_1 = cv2.imread(chan_1_path, -1)
                        chan_2 = cv2.imread(chan_2_path, -1)
                        chan_3 = cv2.imread(chan_3_path, -1)
                        chan_4 = cv2.imread(chan_4_path, -1)
                        
                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                        chan_1_ratio_1 = int(chan_1_int_Q99/chan_1_int_Q01)
                        if chan_1_ratio_1 < 10:
                            chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.999))
                            chan_1_ratio_2 = int(chan_1_int_Q99/chan_1_int_Q01)
                            if chan_1_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2)
                            if chan_1_ratio_2 < 10:
                                chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.001, 0.9999))
                                chan_1_ratio_3 = int(chan_1_int_Q99/chan_1_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2, chan_1_ratio_3)
                        chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.99))
                        chan_2_ratio_1 = int(chan_2_int_Q99/chan_2_int_Q01)
                        if chan_2_ratio_1 < 10:
                            chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.999))
                            chan_2_ratio_2 = int(chan_2_int_Q99/chan_2_int_Q01)
                            if chan_2_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2)
                            if chan_2_ratio_2 < 10:
                                chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.001, 0.9999))
                                chan_2_ratio_3 = int(chan_2_int_Q99/chan_2_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2, chan_2_ratio_3)
                        chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.99))
                        chan_3_ratio_1 = int(chan_3_int_Q99/chan_3_int_Q01)
                        if chan_3_ratio_1 < 10:
                            chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.999))
                            chan_3_ratio_2 = int(chan_3_int_Q99/chan_3_int_Q01)
                            if chan_3_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2)
                            if chan_3_ratio_2 < 10:
                                chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.001, 0.9999))
                                chan_3_ratio_3 = int(chan_3_int_Q99/chan_3_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2, chan_3_ratio_3)
                        chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.99))
                        chan_4_ratio_1 = int(chan_4_int_Q99/chan_4_int_Q01)
                        if chan_4_ratio_1 < 10:
                            chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.999))
                            chan_4_ratio_2 = int(chan_4_int_Q99/chan_4_int_Q01)
                            if chan_4_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2)
                            if chan_4_ratio_2 <10:
                                chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.001, 0.9999))
                                chan_4_ratio_3 = int(chan_4_int_Q99/chan_4_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2, chan_4_ratio_3)
                        
                        stack = np.stack([chan_1, chan_2, chan_3, chan_4], axis=2)
                        print(stack.shape)
                        print([file, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99])
                        new_row = [stack_path, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99]
                        img_intensity_csv = src+'/stack/img_intensity.csv'
                        with open(img_intensity_csv, 'a+', newline='') as write_obj:
                            csv_writer = csv.writer(write_obj)
                            csv_writer.writerow(new_row)
                        print('Stack dtype: '+str(stack.dtype), 'Stack dimensions : '+str(stack.shape))
                        np.save(stack_path, stack.astype(bitdepth))

        if nr_of_channels == 5:
            if os.path.exists(src+'/'+'stack/img_intensity.csv') is False:         
                with open(src+'/'+'stack/img_intensity.csv', 'w') as csvfile:
                    img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99', 'chan_5_int_Q01', 'chan_5_int_Q99'])
            for root, _, files in os.walk(src+'/chan_1'):
                for file in files:
                    chan_1_path=os.path.join(src+'/'+'chan_1', file)
                    chan_2_path=os.path.join(src+'/'+'chan_2', file)
                    chan_3_path=os.path.join(src+'/'+'chan_3', file)
                    chan_4_path=os.path.join(src+'/'+'chan_4', file)
                    chan_5_path=os.path.join(src+'/'+'chan_5', file)
                    stack_path=os.path.join(src+'/'+'stack', file)
                    if os.path.exists(stack_path) == True:
                        print('stack '+file+' exists')
                        pass     
                    else:                    
                        chan_1 = cv2.imread(chan_1_path, -1)
                        chan_2 = cv2.imread(chan_2_path, -1)
                        chan_3 = cv2.imread(chan_3_path, -1)
                        chan_4 = cv2.imread(chan_4_path, -1)
                        chan_5 = cv2.imread(chan_5_path, -1)
                        
                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                        chan_1_ratio_1 = int(chan_1_int_Q99/chan_1_int_Q01)
                        if chan_1_ratio_1 < 10:
                            chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.999))
                            chan_1_ratio_2 = int(chan_1_int_Q99/chan_1_int_Q01)
                            if chan_1_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2)
                            if chan_1_ratio_2 < 10:
                                chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.001, 0.9999))
                                chan_1_ratio_3 = int(chan_1_int_Q99/chan_1_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2, chan_1_ratio_3)
                        chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.99))
                        chan_2_ratio_1 = int(chan_2_int_Q99/chan_2_int_Q01)
                        if chan_2_ratio_1 < 10:
                            chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.999))
                            chan_2_ratio_2 = int(chan_2_int_Q99/chan_2_int_Q01)
                            if chan_2_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2)
                            if chan_2_ratio_2 < 10:
                                chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.001, 0.9999))
                                chan_2_ratio_3 = int(chan_2_int_Q99/chan_2_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2, chan_2_ratio_3)
                        chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.99))
                        chan_3_ratio_1 = int(chan_3_int_Q99/chan_3_int_Q01)
                        if chan_3_ratio_1 < 10:
                            chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.999))
                            chan_3_ratio_2 = int(chan_3_int_Q99/chan_3_int_Q01)
                            if chan_3_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2)
                            if chan_3_ratio_2 < 10:
                                chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.001, 0.9999))
                                chan_3_ratio_3 = int(chan_3_int_Q99/chan_3_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2, chan_3_ratio_3)
                        chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.99))
                        chan_4_ratio_1 = int(chan_4_int_Q99/chan_4_int_Q01)
                        if chan_4_ratio_1 < 10:
                            chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.999))
                            chan_4_ratio_2 = int(chan_4_int_Q99/chan_4_int_Q01)
                            if chan_4_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2)
                            if chan_4_ratio_2 <10:
                                chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.001, 0.9999))
                                chan_4_ratio_3 = int(chan_4_int_Q99/chan_4_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2, chan_4_ratio_3)
                        chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.01, 0.99))
                        chan_5_ratio_1 = int(chan_5_int_Q99/chan_5_int_Q01)
                        if chan_5_ratio_1 < 10:
                            chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.01, 0.999))
                            chan_5_ratio_2 = int(chan_5_int_Q99/chan_5_int_Q01)
                            if chan_5_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_5 Q01/Q99', chan_5_ratio_1, chan_5_ratio_2)
                            if chan_5_ratio_2 <10:
                                chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.001, 0.9999))
                                chan_5_ratio_3 = int(chan_5_int_Q99/chan_5_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_5 Q01/Q99', chan_5_ratio_1, chan_5_ratio_2, chan_5_ratio_3)
                        
                        stack = np.stack([chan_1, chan_2, chan_3, chan_4, chan_5], axis=-1)
                        print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99', 'chan_5_int_Q01', 'chan_5_int_Q99')
                        print([file, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99, chan_5_int_Q01, chan_5_int_Q99])
                        new_row = [stack_path, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99, chan_5_int_Q01, chan_5_int_Q99]
                        img_intensity_csv = src+'/stack/img_intensity.csv'
                        with open(img_intensity_csv, 'a+', newline='') as write_obj:
                            csv_writer = csv.writer(write_obj)
                            csv_writer.writerow(new_row)
                        print('Stack dtype: '+str(stack.dtype), 'Stack dimensions : '+str(stack.shape))
                        np.save(stack_path, stack.astype(bitdepth))
                                                    
        if nr_of_channels == 6:
            if os.path.exists(src+'/'+'stack/img_intensity.csv') is False:         
                with open(src+'/'+'stack/img_intensity.csv', 'w') as csvfile:
                    img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99', 'chan_5_int_Q01', 'chan_5_int_Q99', 'chan_6_int_Q01', 'chan_6_int_Q99'])
            for root, _, files in os.walk(src+'/chan_1'):
                for file in files:
                    chan_1_path=os.path.join(src+'/'+'chan_1', file)
                    chan_2_path=os.path.join(src+'/'+'chan_2', file)
                    chan_3_path=os.path.join(src+'/'+'chan_3', file)
                    chan_4_path=os.path.join(src+'/'+'chan_4', file)
                    chan_5_path=os.path.join(src+'/'+'chan_5', file)
                    chan_6_path=os.path.join(src+'/'+'chan_6', file)
                    stack_path=os.path.join(src+'/'+'stack', file)
                    if os.path.exists(stack_path) == True:
                        print('stack '+file+' exists')
                        pass     
                    else:                    
                        chan_1 = cv2.imread(chan_1_path, -1)
                        chan_2 = cv2.imread(chan_2_path, -1)
                        chan_3 = cv2.imread(chan_3_path, -1)
                        chan_4 = cv2.imread(chan_4_path, -1)
                        chan_5 = cv2.imread(chan_5_path, -1)
                        chan_6 = cv2.imread(chan_6_path, -1)
                        stack = np.stack([chan_1, chan_2, chan_3, chan_4, chan_5, chan_6], axis=-1)
                                                
                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                        chan_1_ratio_1 = int(chan_1_int_Q99/chan_1_int_Q01)
                        if chan_1_ratio_1 < 10:
                            chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.999))
                            chan_1_ratio_2 = int(chan_1_int_Q99/chan_1_int_Q01)
                            if chan_1_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2)
                            if chan_1_ratio_2 < 10:
                                chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.001, 0.9999))
                                chan_1_ratio_3 = int(chan_1_int_Q99/chan_1_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2, chan_1_ratio_3)
                        chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.99))
                        chan_2_ratio_1 = int(chan_2_int_Q99/chan_2_int_Q01)
                        if chan_2_ratio_1 < 10:
                            chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.999))
                            chan_2_ratio_2 = int(chan_2_int_Q99/chan_2_int_Q01)
                            if chan_2_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2)
                            if chan_2_ratio_2 < 10:
                                chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.001, 0.9999))
                                chan_2_ratio_3 = int(chan_2_int_Q99/chan_2_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2, chan_2_ratio_3)
                        chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.99))
                        chan_3_ratio_1 = int(chan_3_int_Q99/chan_3_int_Q01)
                        if chan_3_ratio_1 < 10:
                            chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.999))
                            chan_3_ratio_2 = int(chan_3_int_Q99/chan_3_int_Q01)
                            if chan_3_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2)
                            if chan_3_ratio_2 < 10:
                                chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.001, 0.9999))
                                chan_3_ratio_3 = int(chan_3_int_Q99/chan_3_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2, chan_3_ratio_3)
                        chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.99))
                        chan_4_ratio_1 = int(chan_4_int_Q99/chan_4_int_Q01)
                        if chan_4_ratio_1 < 10:
                            chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.999))
                            chan_4_ratio_2 = int(chan_4_int_Q99/chan_4_int_Q01)
                            if chan_4_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2)
                            if chan_4_ratio_2 <10:
                                chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.001, 0.9999))
                                chan_4_ratio_3 = int(chan_4_int_Q99/chan_4_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2, chan_4_ratio_3)
                        chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.01, 0.99))
                        chan_5_ratio_1 = int(chan_5_int_Q99/chan_5_int_Q01)
                        if chan_5_ratio_1 < 10:
                            chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.01, 0.999))
                            chan_5_ratio_2 = int(chan_5_int_Q99/chan_5_int_Q01)
                            if chan_5_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_5 Q01/Q99', chan_5_ratio_1, chan_5_ratio_2)
                            if chan_5_ratio_2 <10:
                                chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.001, 0.9999))
                                chan_5_ratio_3 = int(chan_5_int_Q99/chan_5_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_5 Q01/Q99', chan_5_ratio_1, chan_5_ratio_2, chan_5_ratio_3)
                        chan_6_int_Q01, chan_6_int_Q99 = np.quantile(chan_6, (.01, 0.99))
                        chan_6_ratio_1 = int(chan_6_int_Q99/chan_6_int_Q01)
                        if chan_6_ratio_1 < 10:
                            chan_6_int_Q01, chan_6_int_Q99 = np.quantile(chan_6, (.01, 0.999))
                            chan_6_ratio_2 = int(chan_6_int_Q99/chan_6_int_Q01)
                            if chan_6_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_6 Q01/Q99', chan_6_ratio_1, chan_6_ratio_2)
                            if chan_6_ratio_2 <10:
                                chan_6_int_Q01, chan_6_int_Q99 = np.quantile(chan_6, (.001, 0.9999))
                                chan_6_ratio_3 = int(chan_6_int_Q99/chan_6_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_6 Q01/Q99', chan_6_ratio_1, chan_6_ratio_2, chan_6_ratio_3)
                        
                        print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99', 'chan_5_int_Q01', 'chan_5_int_Q99', 'chan_6_int_Q01', 'chan_6_int_Q99')
                        print([file, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99, chan_5_int_Q01, chan_5_int_Q99, chan_6_int_Q01, chan_6_int_Q99])
                        new_row = [stack_path, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99, chan_5_int_Q01, chan_5_int_Q99, chan_6_int_Q01, chan_6_int_Q99]
                        img_intensity_csv = src+'/stack/img_intensity.csv'
                        with open(img_intensity_csv, 'a+', newline='') as write_obj:
                            csv_writer = csv.writer(write_obj)
                            csv_writer.writerow(new_row)
                        print('Stack dtype: '+str(stack.dtype), 'Stack dimensions : '+str(stack.shape))
                        np.save(stack_path, stack.astype(bitdepth))
                                                    
        if nr_of_channels == 7:
            if os.path.exists(src+'/'+'stack/img_intensity.csv') is False:         
                with open(src+'/'+'stack/img_intensity.csv', 'w') as csvfile:
                    img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99', 'chan_5_int_Q01', 'chan_5_int_Q99', 'chan_6_int_Q01', 'chan_6_int_Q99', 'chan_7_int_Q01', 'chan_7_int_Q99'])
            for root, _, files in os.walk(src+'/chan_1'):
                for file in files:
                    chan_1_path=os.path.join(src+'/'+'chan_1', file)
                    chan_2_path=os.path.join(src+'/'+'chan_2', file)
                    chan_3_path=os.path.join(src+'/'+'chan_3', file)
                    chan_4_path=os.path.join(src+'/'+'chan_4', file)
                    chan_5_path=os.path.join(src+'/'+'chan_5', file)
                    chan_6_path=os.path.join(src+'/'+'chan_6', file)
                    chan_7_path=os.path.join(src+'/'+'chan_7', file)
                    stack_path=os.path.join(src+'/'+'stack', file)
                    if os.path.exists(stack_path) == True:
                        print('stack '+file+' exists')
                        pass     
                    else:                    
                        chan_1 = cv2.imread(chan_1_path, -1)
                        chan_2 = cv2.imread(chan_2_path, -1)
                        chan_3 = cv2.imread(chan_3_path, -1)
                        chan_4 = cv2.imread(chan_4_path, -1)
                        chan_5 = cv2.imread(chan_5_path, -1)
                        chan_6 = cv2.imread(chan_6_path, -1)
                        chan_7 = cv2.imread(chan_7_path, -1)
                        stack = np.stack([chan_1, chan_2, chan_3, chan_4, chan_5, chan_6, chan_7], axis=-1)
                                                
                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                        chan_1_ratio_1 = int(chan_1_int_Q99/chan_1_int_Q01)
                        if chan_1_ratio_1 < 10:
                            chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.999))
                            chan_1_ratio_2 = int(chan_1_int_Q99/chan_1_int_Q01)
                            if chan_1_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2)
                            if chan_1_ratio_2 < 10:
                                chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.001, 0.9999))
                                chan_1_ratio_3 = int(chan_1_int_Q99/chan_1_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2, chan_1_ratio_3)
                        chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.99))
                        chan_2_ratio_1 = int(chan_2_int_Q99/chan_2_int_Q01)
                        if chan_2_ratio_1 < 10:
                            chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.999))
                            chan_2_ratio_2 = int(chan_2_int_Q99/chan_2_int_Q01)
                            if chan_2_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2)
                            if chan_2_ratio_2 < 10:
                                chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.001, 0.9999))
                                chan_2_ratio_3 = int(chan_2_int_Q99/chan_2_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2, chan_2_ratio_3)
                        chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.99))
                        chan_3_ratio_1 = int(chan_3_int_Q99/chan_3_int_Q01)
                        if chan_3_ratio_1 < 10:
                            chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.999))
                            chan_3_ratio_2 = int(chan_3_int_Q99/chan_3_int_Q01)
                            if chan_3_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2)
                            if chan_3_ratio_2 < 10:
                                chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.001, 0.9999))
                                chan_3_ratio_3 = int(chan_3_int_Q99/chan_3_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2, chan_3_ratio_3)
                        chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.99))
                        chan_4_ratio_1 = int(chan_4_int_Q99/chan_4_int_Q01)
                        if chan_4_ratio_1 < 10:
                            chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.999))
                            chan_4_ratio_2 = int(chan_4_int_Q99/chan_4_int_Q01)
                            if chan_4_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2)
                            if chan_4_ratio_2 <10:
                                chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.001, 0.9999))
                                chan_4_ratio_3 = int(chan_4_int_Q99/chan_4_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2, chan_4_ratio_3)
                        chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.01, 0.99))
                        chan_5_ratio_1 = int(chan_5_int_Q99/chan_5_int_Q01)
                        if chan_5_ratio_1 < 10:
                            chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.01, 0.999))
                            chan_5_ratio_2 = int(chan_5_int_Q99/chan_5_int_Q01)
                            if chan_5_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_5 Q01/Q99', chan_5_ratio_1, chan_5_ratio_2)
                            if chan_5_ratio_2 <10:
                                chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.001, 0.9999))
                                chan_5_ratio_3 = int(chan_5_int_Q99/chan_5_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_5 Q01/Q99', chan_5_ratio_1, chan_5_ratio_2, chan_5_ratio_3)
                        chan_6_int_Q01, chan_6_int_Q99 = np.quantile(chan_6, (.01, 0.99))
                        chan_6_ratio_1 = int(chan_6_int_Q99/chan_6_int_Q01)
                        if chan_6_ratio_1 < 10:
                            chan_6_int_Q01, chan_6_int_Q99 = np.quantile(chan_6, (.01, 0.999))
                            chan_6_ratio_2 = int(chan_6_int_Q99/chan_6_int_Q01)
                            if chan_6_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_6 Q01/Q99', chan_6_ratio_1, chan_6_ratio_2)
                            if chan_6_ratio_2 <10:
                                chan_6_int_Q01, chan_6_int_Q99 = np.quantile(chan_6, (.001, 0.9999))
                                chan_6_ratio_3 = int(chan_6_int_Q99/chan_6_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_6 Q01/Q99', chan_6_ratio_1, chan_6_ratio_2, chan_6_ratio_3)
                        chan_7_int_Q01, chan_7_int_Q99 = np.quantile(chan_7, (.01, 0.99))
                        chan_7_ratio_1 = int(chan_7_int_Q99/chan_7_int_Q01)
                        if chan_7_ratio_1 < 10:
                            chan_7_int_Q01, chan_7_int_Q99 = np.quantile(chan_7, (.01, 0.999))
                            chan_7_ratio_2 = int(chan_7_int_Q99/chan_7_int_Q01)
                            if chan_7_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_7 Q01/Q99', chan_7_ratio_1, chan_7_ratio_2)
                            if chan_7_ratio_2 <10:
                                chan_7_int_Q01, chan_7_int_Q99 = np.quantile(chan_7, (.001, 0.9999))
                                chan_7_ratio_3 = int(chan_7_int_Q99/chan_7_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_7 Q01/Q99', chan_7_ratio_1, chan_7_ratio_2, chan_7_ratio_3)
                        
                        print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99', 'chan_5_int_Q01', 'chan_5_int_Q99', 'chan_6_int_Q01', 'chan_6_int_Q99', 'chan_7_int_Q01', 'chan_7_int_Q99')
                        print([file, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99, chan_5_int_Q01, chan_5_int_Q99, chan_6_int_Q01, chan_6_int_Q99, chan_7_int_Q01, chan_7_int_Q99])
                        new_row = [stack_path, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99, chan_5_int_Q01, chan_5_int_Q99, chan_6_int_Q01, chan_6_int_Q99, chan_7_int_Q01, chan_7_int_Q99]
                        img_intensity_csv = src+'/stack/img_intensity.csv'
                        with open(img_intensity_csv, 'a+', newline='') as write_obj:
                            csv_writer = csv.writer(write_obj)
                            csv_writer.writerow(new_row)
                        print('Stack dtype: '+str(stack.dtype), 'Stack dimensions : '+str(stack.shape))
                        np.save(stack_path, stack.astype(bitdepth))
                                                    
        if nr_of_channels == 8:
            if os.path.exists(src+'/'+'stack/img_intensity.csv') is False:         
                with open(src+'/'+'stack/img_intensity.csv', 'w') as csvfile:
                    img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99', 'chan_5_int_Q01', 'chan_5_int_Q99', 'chan_6_int_Q01', 'chan_6_int_Q99', 'chan_7_int_Q01', 'chan_7_int_Q99', 'chan_8_int_Q01', 'chan_8_int_Q99'])
            for root, _, files in os.walk(src+'/chan_1'):
                for file in files:
                    chan_1_path=os.path.join(src+'/'+'chan_1', file)
                    chan_2_path=os.path.join(src+'/'+'chan_2', file)
                    chan_3_path=os.path.join(src+'/'+'chan_3', file)
                    chan_4_path=os.path.join(src+'/'+'chan_4', file)
                    chan_5_path=os.path.join(src+'/'+'chan_5', file)
                    chan_6_path=os.path.join(src+'/'+'chan_6', file)
                    chan_7_path=os.path.join(src+'/'+'chan_7', file)
                    chan_8_path=os.path.join(src+'/'+'chan_8', file)
                    stack_path=os.path.join(src+'/'+'stack', file)
                    if os.path.exists(stack_path) == True:
                        print('stack '+file+' exists')
                        pass     
                    else:                    
                        chan_1 = cv2.imread(chan_1_path, -1)
                        chan_2 = cv2.imread(chan_2_path, -1)
                        chan_3 = cv2.imread(chan_3_path, -1)
                        chan_4 = cv2.imread(chan_4_path, -1)
                        chan_5 = cv2.imread(chan_5_path, -1)
                        chan_6 = cv2.imread(chan_6_path, -1)
                        chan_7 = cv2.imread(chan_7_path, -1)
                        chan_8 = cv2.imread(chan_8_path, -1)
                        stack = np.stack([chan_1, chan_2, chan_3, chan_4, chan_5, chan_6, chan_7, chan_8], axis=-1)
                                                
                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                        chan_1_ratio_1 = int(chan_1_int_Q99/chan_1_int_Q01)
                        if chan_1_ratio_1 < 10:
                            chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.999))
                            chan_1_ratio_2 = int(chan_1_int_Q99/chan_1_int_Q01)
                            if chan_1_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2)
                            if chan_1_ratio_2 < 10:
                                chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.001, 0.9999))
                                chan_1_ratio_3 = int(chan_1_int_Q99/chan_1_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2, chan_1_ratio_3)
                        chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.99))
                        chan_2_ratio_1 = int(chan_2_int_Q99/chan_2_int_Q01)
                        if chan_2_ratio_1 < 10:
                            chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.999))
                            chan_2_ratio_2 = int(chan_2_int_Q99/chan_2_int_Q01)
                            if chan_2_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2)
                            if chan_2_ratio_2 < 10:
                                chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.001, 0.9999))
                                chan_2_ratio_3 = int(chan_2_int_Q99/chan_2_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2, chan_2_ratio_3)
                        chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.99))
                        chan_3_ratio_1 = int(chan_3_int_Q99/chan_3_int_Q01)
                        if chan_3_ratio_1 < 10:
                            chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.999))
                            chan_3_ratio_2 = int(chan_3_int_Q99/chan_3_int_Q01)
                            if chan_3_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2)
                            if chan_3_ratio_2 < 10:
                                chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.001, 0.9999))
                                chan_3_ratio_3 = int(chan_3_int_Q99/chan_3_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2, chan_3_ratio_3)
                        chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.99))
                        chan_4_ratio_1 = int(chan_4_int_Q99/chan_4_int_Q01)
                        if chan_4_ratio_1 < 10:
                            chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.999))
                            chan_4_ratio_2 = int(chan_4_int_Q99/chan_4_int_Q01)
                            if chan_4_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2)
                            if chan_4_ratio_2 <10:
                                chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.001, 0.9999))
                                chan_4_ratio_3 = int(chan_4_int_Q99/chan_4_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2, chan_4_ratio_3)
                        chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.01, 0.99))
                        chan_5_ratio_1 = int(chan_5_int_Q99/chan_5_int_Q01)
                        if chan_5_ratio_1 < 10:
                            chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.01, 0.999))
                            chan_5_ratio_2 = int(chan_5_int_Q99/chan_5_int_Q01)
                            if chan_5_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_5 Q01/Q99', chan_5_ratio_1, chan_5_ratio_2)
                            if chan_5_ratio_2 <10:
                                chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.001, 0.9999))
                                chan_5_ratio_3 = int(chan_5_int_Q99/chan_5_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_5 Q01/Q99', chan_5_ratio_1, chan_5_ratio_2, chan_5_ratio_3)
                        chan_6_int_Q01, chan_6_int_Q99 = np.quantile(chan_6, (.01, 0.99))
                        chan_6_ratio_1 = int(chan_6_int_Q99/chan_6_int_Q01)
                        if chan_6_ratio_1 < 10:
                            chan_6_int_Q01, chan_6_int_Q99 = np.quantile(chan_6, (.01, 0.999))
                            chan_6_ratio_2 = int(chan_6_int_Q99/chan_6_int_Q01)
                            if chan_6_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_6 Q01/Q99', chan_6_ratio_1, chan_6_ratio_2)
                            if chan_6_ratio_2 <10:
                                chan_6_int_Q01, chan_6_int_Q99 = np.quantile(chan_6, (.001, 0.9999))
                                chan_6_ratio_3 = int(chan_6_int_Q99/chan_6_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_6 Q01/Q99', chan_6_ratio_1, chan_6_ratio_2, chan_6_ratio_3)
                        chan_7_int_Q01, chan_7_int_Q99 = np.quantile(chan_7, (.01, 0.99))
                        chan_7_ratio_1 = int(chan_7_int_Q99/chan_7_int_Q01)
                        if chan_7_ratio_1 < 10:
                            chan_7_int_Q01, chan_7_int_Q99 = np.quantile(chan_7, (.01, 0.999))
                            chan_7_ratio_2 = int(chan_7_int_Q99/chan_7_int_Q01)
                            if chan_7_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_7 Q01/Q99', chan_7_ratio_1, chan_7_ratio_2)
                            if chan_7_ratio_2 <10:
                                chan_7_int_Q01, chan_7_int_Q99 = np.quantile(chan_7, (.001, 0.9999))
                                chan_7_ratio_3 = int(chan_7_int_Q99/chan_7_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_7 Q01/Q99', chan_7_ratio_1, chan_7_ratio_2, chan_7_ratio_3)
                        chan_8_int_Q01, chan_8_int_Q99 = np.quantile(chan_8, (.01, 0.99))
                        chan_8_ratio_1 = int(chan_8_int_Q99/chan_8_int_Q01)
                        if chan_8_ratio_1 < 10:
                            chan_8_int_Q01, chan_8_int_Q99 = np.quantile(chan_8, (.01, 0.999))
                            chan_8_ratio_2 = int(chan_8_int_Q99/chan_8_int_Q01)
                            if chan_8_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_8 Q01/Q99', chan_8_ratio_1, chan_8_ratio_2)
                            if chan_8_ratio_2 <10:
                                chan_8_int_Q01, chan_8_int_Q99 = np.quantile(chan_8, (.001, 0.9999))
                                chan_8_ratio_3 = int(chan_8_int_Q99/chan_8_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_8 Q01/Q99', chan_8_ratio_1, chan_8_ratio_2, chan_8_ratio_3)
                        
                        print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99', 'chan_5_int_Q01', 'chan_5_int_Q99', 'chan_6_int_Q01', 'chan_6_int_Q99', 'chan_7_int_Q01', 'chan_7_int_Q99', 'chan_8_int_Q01', 'chan_8_int_Q99')
                        print([file, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99, chan_5_int_Q01, chan_5_int_Q99, chan_6_int_Q01, chan_6_int_Q99, chan_7_int_Q01, chan_7_int_Q99, chan_8_int_Q01, chan_8_int_Q99])
                        new_row = [stack_path, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99, chan_5_int_Q01, chan_5_int_Q99, chan_6_int_Q01, chan_6_int_Q99, chan_7_int_Q01, chan_7_int_Q99, chan_8_int_Q01, chan_8_int_Q99]
                        img_intensity_csv = src+'/stack/img_intensity.csv'
                        with open(img_intensity_csv, 'a+', newline='') as write_obj:
                            csv_writer = csv.writer(write_obj)
                            csv_writer.writerow(new_row)
                        print('Stack dtype: '+str(stack.dtype), 'Stack dimensions : '+str(stack.shape))
                        np.save(stack_path, stack.astype(bitdepth))
                        
        if nr_of_channels == 9:
            if os.path.exists(src+'/'+'stack/img_intensity.csv') is False:         
                with open(src+'/'+'stack/img_intensity.csv', 'w') as csvfile:
                    img_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    img_intensity.writerow(['chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99', 'chan_5_int_Q01', 'chan_5_int_Q99', 'chan_6_int_Q01', 'chan_6_int_Q99', 'chan_7_int_Q01', 'chan_7_int_Q99', 'chan_8_int_Q01', 'chan_8_int_Q99', 'chan_9_int_Q01', 'chan_9_int_Q99'])
            for root, _, files in os.walk(src+'/chan_1'):
                for file in files:
                    chan_1_path=os.path.join(src+'/'+'chan_1', file)
                    chan_2_path=os.path.join(src+'/'+'chan_2', file)
                    chan_3_path=os.path.join(src+'/'+'chan_3', file)
                    chan_4_path=os.path.join(src+'/'+'chan_4', file)
                    chan_5_path=os.path.join(src+'/'+'chan_5', file)
                    chan_6_path=os.path.join(src+'/'+'chan_6', file)
                    chan_7_path=os.path.join(src+'/'+'chan_7', file)
                    chan_8_path=os.path.join(src+'/'+'chan_8', file)
                    chan_9_path=os.path.join(src+'/'+'chan_9', file)
                    stack_path=os.path.join(src+'/'+'stack', file)
                    if os.path.exists(stack_path) == True:
                        print('stack '+file+' exists')
                        pass     
                    else:    
                        chan_1 = cv2.imread(chan_1_path, -1)
                        chan_2 = cv2.imread(chan_2_path, -1)
                        chan_3 = cv2.imread(chan_3_path, -1)
                        chan_4 = cv2.imread(chan_4_path, -1)
                        chan_5 = cv2.imread(chan_5_path, -1)
                        chan_6 = cv2.imread(chan_6_path, -1)
                        chan_7 = cv2.imread(chan_7_path, -1)
                        chan_8 = cv2.imread(chan_8_path, -1)
                        chan_9 = cv2.imread(chan_9_path, -1)
                        stack = np.stack([chan_1, chan_2, chan_3, chan_4, chan_5, chan_6, chan_7, chan_8, chan_9], axis=-1)
                                                
                        chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.99))
                        chan_1_ratio_1 = int(chan_1_int_Q99/chan_1_int_Q01)
                        if chan_1_ratio_1 < 10:
                            chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.01, 0.999))
                            chan_1_ratio_2 = int(chan_1_int_Q99/chan_1_int_Q01)
                            if chan_1_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2)
                            if chan_1_ratio_2 < 10:
                                chan_1_int_Q01, chan_1_int_Q99 = np.quantile(chan_1, (.001, 0.9999))
                                chan_1_ratio_3 = int(chan_1_int_Q99/chan_1_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_1 Q01/Q99', chan_1_ratio_1, chan_1_ratio_2, chan_1_ratio_3)
                        chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.99))
                        chan_2_ratio_1 = int(chan_2_int_Q99/chan_2_int_Q01)
                        if chan_2_ratio_1 < 10:
                            chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.01, 0.999))
                            chan_2_ratio_2 = int(chan_2_int_Q99/chan_2_int_Q01)
                            if chan_2_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2)
                            if chan_2_ratio_2 < 10:
                                chan_2_int_Q01, chan_2_int_Q99 = np.quantile(chan_2, (.001, 0.9999))
                                chan_2_ratio_3 = int(chan_2_int_Q99/chan_2_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_2 Q01/Q99', chan_2_ratio_1, chan_2_ratio_2, chan_2_ratio_3)
                        chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.99))
                        chan_3_ratio_1 = int(chan_3_int_Q99/chan_3_int_Q01)
                        if chan_3_ratio_1 < 10:
                            chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.01, 0.999))
                            chan_3_ratio_2 = int(chan_3_int_Q99/chan_3_int_Q01)
                            if chan_3_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2)
                            if chan_3_ratio_2 < 10:
                                chan_3_int_Q01, chan_3_int_Q99 = np.quantile(chan_3, (.001, 0.9999))
                                chan_3_ratio_3 = int(chan_3_int_Q99/chan_3_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_3 Q01/Q99', chan_3_ratio_1, chan_3_ratio_2, chan_3_ratio_3)
                        chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.99))
                        chan_4_ratio_1 = int(chan_4_int_Q99/chan_4_int_Q01)
                        if chan_4_ratio_1 < 10:
                            chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.01, 0.999))
                            chan_4_ratio_2 = int(chan_4_int_Q99/chan_4_int_Q01)
                            if chan_4_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2)
                            if chan_4_ratio_2 <10:
                                chan_4_int_Q01, chan_4_int_Q99 = np.quantile(chan_4, (.001, 0.9999))
                                chan_4_ratio_3 = int(chan_4_int_Q99/chan_4_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_4 Q01/Q99', chan_4_ratio_1, chan_4_ratio_2, chan_4_ratio_3)
                        chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.01, 0.99))
                        chan_5_ratio_1 = int(chan_5_int_Q99/chan_5_int_Q01)
                        if chan_5_ratio_1 < 10:
                            chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.01, 0.999))
                            chan_5_ratio_2 = int(chan_5_int_Q99/chan_5_int_Q01)
                            if chan_5_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_5 Q01/Q99', chan_5_ratio_1, chan_5_ratio_2)
                            if chan_5_ratio_2 <10:
                                chan_5_int_Q01, chan_5_int_Q99 = np.quantile(chan_5, (.001, 0.9999))
                                chan_5_ratio_3 = int(chan_5_int_Q99/chan_5_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_5 Q01/Q99', chan_5_ratio_1, chan_5_ratio_2, chan_5_ratio_3)
                        chan_6_int_Q01, chan_6_int_Q99 = np.quantile(chan_6, (.01, 0.99))
                        chan_6_ratio_1 = int(chan_6_int_Q99/chan_6_int_Q01)
                        if chan_6_ratio_1 < 10:
                            chan_6_int_Q01, chan_6_int_Q99 = np.quantile(chan_6, (.01, 0.999))
                            chan_6_ratio_2 = int(chan_6_int_Q99/chan_6_int_Q01)
                            if chan_6_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_6 Q01/Q99', chan_6_ratio_1, chan_6_ratio_2)
                            if chan_6_ratio_2 <10:
                                chan_6_int_Q01, chan_6_int_Q99 = np.quantile(chan_6, (.001, 0.9999))
                                chan_6_ratio_3 = int(chan_6_int_Q99/chan_6_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_6 Q01/Q99', chan_6_ratio_1, chan_6_ratio_2, chan_6_ratio_3)
                        chan_7_int_Q01, chan_7_int_Q99 = np.quantile(chan_7, (.01, 0.99))
                        chan_7_ratio_1 = int(chan_7_int_Q99/chan_7_int_Q01)
                        if chan_7_ratio_1 < 10:
                            chan_7_int_Q01, chan_7_int_Q99 = np.quantile(chan_7, (.01, 0.999))
                            chan_7_ratio_2 = int(chan_7_int_Q99/chan_7_int_Q01)
                            if chan_7_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_7 Q01/Q99', chan_7_ratio_1, chan_7_ratio_2)
                            if chan_7_ratio_2 <10:
                                chan_7_int_Q01, chan_7_int_Q99 = np.quantile(chan_7, (.001, 0.9999))
                                chan_7_ratio_3 = int(chan_7_int_Q99/chan_7_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_7 Q01/Q99', chan_7_ratio_1, chan_7_ratio_2, chan_7_ratio_3)
                        chan_8_int_Q01, chan_8_int_Q99 = np.quantile(chan_8, (.01, 0.99))
                        chan_8_ratio_1 = int(chan_8_int_Q99/chan_8_int_Q01)
                        if chan_8_ratio_1 < 10:
                            chan_8_int_Q01, chan_8_int_Q99 = np.quantile(chan_8, (.01, 0.999))
                            chan_8_ratio_2 = int(chan_8_int_Q99/chan_8_int_Q01)
                            if chan_8_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_8 Q01/Q99', chan_8_ratio_1, chan_8_ratio_2)
                            if chan_8_ratio_2 <10:
                                chan_8_int_Q01, chan_8_int_Q99 = np.quantile(chan_8, (.001, 0.9999))
                                chan_8_ratio_3 = int(chan_8_int_Q99/chan_8_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_8 Q01/Q99', chan_8_ratio_1, chan_8_ratio_2, chan_8_ratio_3)
                        chan_9_int_Q01, chan_9_int_Q99 = np.quantile(chan_9, (.01, 0.99))
                        chan_9_ratio_1 = int(chan_9_int_Q99/chan_9_int_Q01)
                        if chan_9_ratio_1 < 10:
                            chan_9_int_Q01, chan_9_int_Q99 = np.quantile(chan_9, (.01, 0.999))
                            chan_9_ratio_2 = int(chan_9_int_Q99/chan_9_int_Q01)
                            if chan_9_ratio_2 >= 10:
                                print('WARNING: Q99 set to 0.999 for: chan_9 Q01/Q99', chan_9_ratio_1, chan_9_ratio_2)
                            if chan_9_ratio_2 <10:
                                chan_9_int_Q01, chan_9_int_Q99 = np.quantile(chan_9, (.001, 0.9999))
                                chan_9_ratio_3 = int(chan_9_int_Q99/chan_9_int_Q01)
                                print('WARNING: Q01 set to 0.001 Q99 set to 0.9999 for: chan_9 Q01/Q99', chan_9_ratio_1, chan_9_ratio_2, chan_9_ratio_3)
                        
                        print('file name', 'chan_1_int_Q01', 'chan_1_int_Q99', 'chan_2_int_Q01', 'chan_2_int_Q99', 'chan_3_int_Q01', 'chan_3_int_Q99', 'chan_4_int_Q01', 'chan_4_int_Q99', 'chan_5_int_Q01', 'chan_5_int_Q99', 'chan_6_int_Q01', 'chan_6_int_Q99', 'chan_7_int_Q01', 'chan_7_int_Q99', 'chan_8_int_Q01', 'chan_8_int_Q99', 'chan_9_int_Q01', 'chan_9_int_Q99')
                        print([file, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99, chan_5_int_Q01, chan_5_int_Q99, chan_6_int_Q01, chan_6_int_Q99, chan_7_int_Q01, chan_7_int_Q99, chan_8_int_Q01, chan_8_int_Q99, chan_9_int_Q01, chan_9_int_Q99])
                        new_row = [stack_path, chan_1_int_Q01, chan_1_int_Q99, chan_2_int_Q01, chan_2_int_Q99, chan_3_int_Q01, chan_3_int_Q99, chan_4_int_Q01, chan_4_int_Q99, chan_5_int_Q01, chan_5_int_Q99, chan_6_int_Q01, chan_6_int_Q99, chan_7_int_Q01, chan_7_int_Q99, chan_8_int_Q01, chan_8_int_Q99, chan_9_int_Q01, chan_9_int_Q99]
                        img_intensity_csv = src+'/stack/img_intensity.csv'
                        with open(img_intensity_csv, 'a+', newline='') as write_obj:
                            csv_writer = csv.writer(write_obj)
                            csv_writer.writerow(new_row)               
                        print('Stack dtype: '+str(stack.dtype), 'Stack dimensions : '+str(stack.shape))
                        cv2.imwrite(stack_path, stack.astype(bitdepth))


#This Function prinsts each image and channel in a stack.                 
def plot_arrays_in_stack(stack):
    stack_dim = stack.shape
    channel = stack_dim[2]
    image_dim = stack_dim[3]
    if len(stack_dim)==4:
        a, b, c = np.split(stack, 3, axis=3)
        if a.shape == b.shape == c.shape:
            n_dim=len(a.shape)
        
            dim=a.shape #dimensions of image
            channel_position=dim.index(channel)
            
            print('Stack dimensions: ', stack_dim)
            print('Image index: ', image_dim)
            print('Channel index: ', channel_position)
            print("img has "+str(n_dim)+" dimensions")
            channel_image_a=a.shape[2]
            channel_image_b=b.shape[2]
            channel_image_c=c.shape[2]
            
            fig, axs = plt.subplots(1, channel_image_a,figsize=(20,20))
            
            for channel in range(channel_image_a):
                axs[channel].imshow(a[:,:,channel])
                axs[channel].set_title('Slice '+str(channel+1),size=18)
                axs[channel].axis('off')
            fig, axs = plt.subplots(1, channel_image_b,figsize=(20,20))
            for channel in range(channel_image_b):
                axs[channel].imshow(b[:,:,channel])
                axs[channel].set_title('Slice '+str(channel+3),size=18)
                axs[channel].axis('off')
            fig, axs = plt.subplots(1, channel_image_c,figsize=(20,20))
            for channel in range(channel_image_c):
                axs[channel].imshow(c[:,:,channel])
                axs[channel].set_title('Slice '+str(channel+6),size=18)
                axs[channel].axis('off')
                
# explicit function to normalize array
def normalize_array(arr, chan):
    norm_arr = []
    diff = 10000 - 1
    if chan == '1':
        diff_arr = chan_1_int_Q99 - chan_1_int_Q5
        Q5 = chan_1_int_Q5
    if chan == '2':
        diff_arr = chan_2_int_Q99 - chan_2_int_Q5
        Q5 = chan_2_int_Q5
    if chan == '3':
        diff_arr = chan_3_int_Q99 - chan_3_int_Q5
        Q5 = chan_3_int_Q5
    if chan == '4':
        diff_arr = chan_4_int_Q99 - chan_4_int_Q5
        Q5 = chan_4_int_Q5
    if chan == '5':
        diff_arr = chan_5_int_Q99 - chan_5_int_Q5
        Q5 = chan_5_int_Q5
    if chan == '6':
        diff_arr = chan_6_int_Q99 - chan_6_int_Q5
        Q5 = chan_6_int_Q5
    if chan == '7':
        diff_arr = chan_7_int_Q99 - chan_7_int_Q5
        Q5 = chan_7_int_Q5
    if chan == '8':
        diff_arr = chan_8_int_Q99 - chan_8_int_Q5
        Q5 = chan_8_int_Q5
    if chan == '9':
        diff_arr = chan_9_int_Q99 - chan_9_int_Q5
        Q5 = chan_9_int_Q5
    if environ.get('Q5') is not None: 
        for i in arr:
            temp = (((i - Q5)*diff)/diff_arr) + Q5
            norm_arr.append(temp)
    #return norm_arr
    
#This Function uses the intensity quantiles colected in 'gray_to_color' and normalizes
#images on a per plate and per channel bases to the respective quantiles. 
def rescal_intensity(src, nr_of_channels, mode):
    if environ.get('chan_1_upperQnr2') is None:
        chan_1_upperQnr2 = 0.99
    if environ.get('chan_2_upperQnr2') is None:
        chan_2_upperQnr2 = 0.99
    if environ.get('chan_3_upperQnr2') is None:
        chan_3_upperQnr2 = 0.99
    if environ.get('chan_4_upperQnr2') is None:
        chan_4_upperQnr2 = 0.99
    if environ.get('chan_5_upperQnr2') is None:
        chan_5_upperQnr2 = 0.99
    if environ.get('chan_6_upperQnr2') is None:
        chan_6_upperQnr2 = 0.99
    if environ.get('chan_7_upperQnr2') is None:
        chan_7_upperQnr2 = 0.99
    if environ.get('chan_8_upperQnr2') is None:
        chan_8_upperQnr2 = 0.99
    if environ.get('chan_9_upperQnr2') is None:
        chan_9_upperQnr2 = 0.99
    
    if os.path.exists(src+'/'+'gray') is True:
        mode = 'gray'
        img_intensity_csv = src+'/gray/img_intensity.csv'
        if os.path.exists(src+'/'+'gray_orig') is False:         
            os.mkdir(src+'/'+'gray_orig')
                                          
    if os.path.exists(src+'/'+'rgb') is True:
        mode = 'rgb'
        img_intensity_csv = src+'/rgb/img_intensity.csv'
        if os.path.exists(src+'/'+'rgb_orig') is False:         
            os.mkdir(src+'/'+'rgb_orig')
                                       
    if os.path.exists(src+'/'+'rgba') is True:
        mode = 'rgba'
        img_intensity_csv = src+'/rgba/img_intensity.csv'
        if os.path.exists(src+'/'+'rgba_orig') is False:         
            os.mkdir(src+'/'+'rgba_orig')
                       
    if os.path.exists(src+'/'+'stack') is True: 
        mode = 'stack'
        img_intensity_csv = src+'/stack/img_intensity.csv'
        if os.path.exists(src+'/'+'stack_orig') is False:         
            os.mkdir(src+'/'+'stack_orig')
        
    df = pd.read_csv (img_intensity_csv)
                                          
    if nr_of_channels == 1:
        chan_1_int_Q5 = df['chan_1_int_Q01'].quantile(0.5)
        chan_1_int_Q99 = df['chan_1_int_Q99'].quantile(chan_1_upperQnr2)
        print('file name', 'chan_1_int_Q5', 'chan_1_int_Q99')
    if nr_of_channels == 2:
        chan_1_int_Q5 = df['chan_1_int_Q01'].quantile(0.5)
        chan_1_int_Q99 = df['chan_1_int_Q99'].quantile(chan_1_upperQnr2)
        chan_2_int_Q5 = df['chan_2_int_Q01'].quantile(0.5)
        chan_2_int_Q99 = df['chan_2_int_Q99'].quantile(chan_2_upperQnr2)
        print('file name', 'chan_1_int_Q5', 'chan_1_int_Q99', 'chan_2_int_Q5', 'chan_2_int_Q99')
    if nr_of_channels == 3:
        chan_1_int_Q5 = df['chan_1_int_Q01'].quantile(0.5)
        chan_1_int_Q99 = df['chan_1_int_Q99'].quantile(chan_1_upperQnr2)
        chan_2_int_Q5 = df['chan_2_int_Q01'].quantile(0.5)
        chan_2_int_Q99 = df['chan_2_int_Q99'].quantile(chan_2_upperQnr2)
        chan_3_int_Q5 = df['chan_3_int_Q01'].quantile(0.5)
        chan_3_int_Q99 = df['chan_3_int_Q99'].quantile(chan_3_upperQnr2)
        print('file name', 'chan_1_int_Q5', 'chan_1_int_Q99', 'chan_2_int_Q5', 'chan_2_int_Q99', 'chan_3_int_Q5', 'chan_3_Q99')                                          
    if nr_of_channels == 4:
        chan_1_int_Q5 = df['chan_1_int_Q01'].quantile(0.5)
        chan_1_int_Q99 = df['chan_1_int_Q99'].quantile(chan_1_upperQnr2)
        chan_2_int_Q5 = df['chan_2_int_Q01'].quantile(0.5)
        chan_2_int_Q99 = df['chan_2_int_Q99'].quantile(chan_2_upperQnr2)
        chan_3_int_Q5 = df['chan_3_int_Q01'].quantile(0.5)
        chan_3_int_Q99 = df['chan_3_int_Q99'].quantile(chan_3_upperQnr2)
        chan_4_int_Q5 = df['chan_4_int_Q01'].quantile(0.5)
        chan_4_int_Q99 = df['chan_4_int_Q99'].quantile(chan_4_upperQnr2)
        
        chan_1_signal_to_nois = chan_1_int_Q99/chan_1_int_Q5
        if chan_1_signal_to_nois < 10:
            print('WARNING: Increasing upper intensity quantile to 0.999')
            chan_1_upperQnr2=0.999
            chan_1_int_Q99 = df['chan_1_int_Q99'].quantile(chan_1_upperQnr2)
        
        chan_2_signal_to_nois = chan_2_int_Q99/chan_2_int_Q5
        if chan_2_signal_to_nois < 10:
            print('WARNING: Increasing upper intensity quantile to 0.999')
            chan_2_upperQnr2=0.999
            chan_2_int_Q99 = df['chan_2_int_Q99'].quantile(chan_2_upperQnr2)
        
        chan_3_signal_to_nois = chan_3_int_Q99/chan_3_int_Q5
        if chan_3_signal_to_nois < 10:
            print('WARNING: Increasing upper intensity quantile to 0.999')
            chan_3_upperQnr2=0.999
            chan_3_int_Q99 = df['chan_3_int_Q99'].quantile(chan_3_upperQnr2)
        
        chan_4_signal_to_nois = chan_4_int_Q99/chan_4_int_Q5
        if chan_4_signal_to_nois < 10:
            print('WARNING: Increasing upper intensity quantile to 0.999')
            chan_4_upperQnr2=0.999
            chan_4_int_Q99 = df['chan_4_int_Q99'].quantile(chan_4_upperQnr2)
        
        chan_1_signal_to_nois = chan_1_int_Q99/chan_1_int_Q5
        if chan_1_signal_to_nois < 10:
            print('WARNING: forground/background < 10: chan_1')
            
        chan_2_signal_to_nois = chan_2_int_Q99/chan_2_int_Q5
        if chan_1_signal_to_nois < 10:
            print('WARNING: forground/background < 10: chan_2')
            
        chan_3_signal_to_nois = chan_3_int_Q99/chan_3_int_Q5
        if chan_1_signal_to_nois < 10:
            print('WARNING: forground/background < 10: chan_3')
            
        chan_4_signal_to_nois = chan_4_int_Q99/chan_4_int_Q5
        if chan_1_signal_to_nois < 10:
            print('WARNING: forground/background < 10: chan_4')
            
        print(chan_1_upperQnr2, chan_2_upperQnr2, chan_3_upperQnr2, chan_4_upperQnr2)
        print('file name', 'chan_1_int_Q5', 'chan_1_int_Q99', 'chan_2_int_Q5', 'chan_2_int_Q99', 'chan_3_int_Q5', 'chan_3_Q99', 'chan_4_int_Q5', 'chan_4_int_Q99')                                          
    if nr_of_channels == 5:
        chan_1_int_Q5 = df['chan_1_int_Q01'].quantile(0.5)
        chan_1_int_Q99 = df['chan_1_int_Q99'].quantile(chan_1_upperQnr2)
        chan_2_int_Q5 = df['chan_2_int_Q01'].quantile(0.5)
        chan_2_int_Q99 = df['chan_2_int_Q99'].quantile(chan_2_upperQnr2)
        chan_3_int_Q5 = df['chan_3_int_Q01'].quantile(0.5)
        chan_3_int_Q99 = df['chan_3_int_Q99'].quantile(chan_3_upperQnr2)
        chan_4_int_Q5 = df['chan_4_int_Q01'].quantile(0.5)
        chan_4_int_Q99 = df['chan_4_int_Q99'].quantile(chan_4_upperQnr2)
        chan_5_int_Q5 = df['chan_5_int_Q01'].quantile(0.5)
        chan_5_int_Q99 = df['chan_5_int_Q99'].quantile(chan_5_upperQnr2)
        print('file name', 'chan_2_int_Q5', 'chan_2_int_Q99', 'chan_3_int_Q5', 'chan_3_Q99', 'chan_4_int_Q5', 'chan_4_int_Q99', 'chan_5_int_Q5', 'chan_5_int_Q99')
    if nr_of_channels == 6:
        chan_1_int_Q5 = df['chan_1_int_Q01'].quantile(0.5)
        chan_1_int_Q99 = df['chan_1_int_Q99'].quantile(chan_1_upperQnr2)
        chan_2_int_Q5 = df['chan_2_int_Q01'].quantile(0.5)
        chan_2_int_Q99 = df['chan_2_int_Q99'].quantile(chan_2_upperQnr2)
        chan_3_int_Q5 = df['chan_3_int_Q01'].quantile(0.5)
        chan_3_int_Q99 = df['chan_3_int_Q99'].quantile(chan_3_upperQnr2)
        chan_4_int_Q5 = df['chan_4_int_Q01'].quantile(0.5)
        chan_4_int_Q99 = df['chan_4_int_Q99'].quantile(chan_4_upperQnr2)
        chan_5_int_Q5 = df['chan_5_int_Q01'].quantile(0.5)
        chan_5_int_Q99 = df['chan_5_int_Q99'].quantile(chan_5_upperQnr2)
        chan_6_int_Q5 = df['chan_6_int_Q01'].quantile(0.5)
        chan_6_int_Q99 = df['chan_6_int_Q99'].quantile(chan_6_upperQnr2)
        print('file name', 'chan_2_int_Q5', 'chan_2_int_Q99', 'chan_3_int_Q5', 'chan_3_Q99', 'chan_4_int_Q5', 'chan_4_int_Q99', 'chan_5_int_Q5', 'chan_5_int_Q99', 'chan_6_int_Q5', 'chan_6_int_Q99')                                          
    if nr_of_channels == 7:
        chan_1_int_Q5 = df['chan_1_int_Q01'].quantile(0.5)
        chan_1_int_Q99 = df['chan_1_int_Q99'].quantile(chan_1_upperQnr2)
        chan_2_int_Q5 = df['chan_2_int_Q01'].quantile(0.5)
        chan_2_int_Q99 = df['chan_2_int_Q99'].quantile(chan_2_upperQnr2)
        chan_3_int_Q5 = df['chan_3_int_Q01'].quantile(0.5)
        chan_3_int_Q99 = df['chan_3_int_Q99'].quantile(chan_3_upperQnr2)
        chan_4_int_Q5 = df['chan_4_int_Q01'].quantile(0.5)
        chan_4_int_Q99 = df['chan_4_int_Q99'].quantile(chan_4_upperQnr2)
        chan_5_int_Q5 = df['chan_5_int_Q01'].quantile(0.5)
        chan_5_int_Q99 = df['chan_5_int_Q99'].quantile(chan_5_upperQnr2)
        chan_6_int_Q5 = df['chan_6_int_Q01'].quantile(0.5)
        chan_6_int_Q99 = df['chan_6_int_Q99'].quantile(chan_6_upperQnr2)
        chan_7_int_Q5 = df['chan_7_int_Q01'].quantile(0.5)
        chan_7_int_Q99 = df['chan_7_int_Q99'].quantile(chan_7_upperQnr2)
        print('file name', 'chan_2_int_Q5', 'chan_2_int_Q99', 'chan_3_int_Q5', 'chan_3_Q99', 'chan_4_int_Q5', 'chan_4_int_Q99', 'chan_5_int_Q5', 'chan_5_int_Q99', 'chan_6_int_Q5', 'chan_6_int_Q99', 'chan_7_int_Q5', 'chan_7_int_Q99')                                          
    if nr_of_channels == 8:
        chan_1_int_Q5 = df['chan_1_int_Q01'].quantile(0.5)
        chan_1_int_Q99 = df['chan_1_int_Q99'].quantile(chan_1_upperQnr2)
        chan_2_int_Q5 = df['chan_2_int_Q01'].quantile(0.5)
        chan_2_int_Q99 = df['chan_2_int_Q99'].quantile(chan_2_upperQnr2)
        chan_3_int_Q5 = df['chan_3_int_Q01'].quantile(0.5)
        chan_3_int_Q99 = df['chan_3_int_Q99'].quantile(chan_3_upperQnr2)
        chan_4_int_Q5 = df['chan_4_int_Q01'].quantile(0.5)
        chan_4_int_Q99 = df['chan_4_int_Q99'].quantile(chan_4_upperQnr2)
        chan_5_int_Q5 = df['chan_5_int_Q01'].quantile(0.5)
        chan_5_int_Q99 = df['chan_5_int_Q99'].quantile(chan_5_upperQnr2)
        chan_6_int_Q5 = df['chan_6_int_Q01'].quantile(0.5)
        chan_6_int_Q99 = df['chan_6_int_Q99'].quantile(chan_6_upperQnr2)
        chan_7_int_Q5 = df['chan_7_int_Q01'].quantile(0.5)
        chan_7_int_Q99 = df['chan_7_int_Q99'].quantile(chan_7_upperQnr2)
        chan_8_int_Q5 = df['chan_8_int_Q01'].quantile(0.5)
        chan_8_int_Q99 = df['chan_8_int_Q99'].quantile(chan_8_upperQnr2)
        print('file name', 'chan_2_int_Q5', 'chan_2_int_Q99', 'chan_3_int_Q5', 'chan_3_Q99', 'chan_4_int_Q5', 'chan_4_int_Q99', 'chan_5_int_Q5', 'chan_5_int_Q99', 'chan_6_int_Q5', 'chan_6_int_Q99', 'chan_7_int_Q5', 'chan_7_int_Q99', 'chan_8_int_Q5', 'chan_8_int_Q99')
    if nr_of_channels == 9:
        chan_1_int_Q5 = df['chan_1_int_Q01'].quantile(0.5)
        chan_1_int_Q99 = df['chan_1_int_Q99'].quantile(chan_1_upperQnr2)
        chan_2_int_Q5 = df['chan_2_int_Q01'].quantile(0.5)
        chan_2_int_Q99 = df['chan_2_int_Q99'].quantile(chan_2_upperQnr2)
        chan_3_int_Q5 = df['chan_3_int_Q01'].quantile(0.5)
        chan_3_int_Q99 = df['chan_3_int_Q99'].quantile(chan_3_upperQnr2)
        chan_4_int_Q5 = df['chan_4_int_Q01'].quantile(0.5)
        chan_4_int_Q99 = df['chan_4_int_Q99'].quantile(chan_4_upperQnr2)
        chan_5_int_Q5 = df['chan_5_int_Q01'].quantile(0.5)
        chan_5_int_Q99 = df['chan_5_int_Q99'].quantile(chan_5_upperQnr2)
        chan_6_int_Q5 = df['chan_6_int_Q01'].quantile(0.5)
        chan_6_int_Q99 = df['chan_6_int_Q99'].quantile(chan_6_upperQnr2)
        chan_7_int_Q5 = df['chan_7_int_Q01'].quantile(0.5)
        chan_7_int_Q99 = df['chan_7_int_Q99'].quantile(chan_7_upperQnr2)
        chan_8_int_Q5 = df['chan_8_int_Q01'].quantile(0.5)
        chan_8_int_Q99 = df['chan_8_int_Q99'].quantile(chan_8_upperQnr2)
        chan_9_int_Q5 = df['chan_9_int_Q01'].quantile(0.5)
        chan_9_int_Q99 = df['chan_9_int_Q99'].quantile(chan_9_upperQnr2)
        print('file name', 'chan_2_int_Q5', 'chan_2_int_Q99', 'chan_3_int_Q5', 'chan_3_Q99', 'chan_4_int_Q5', 'chan_4_int_Q99', 'chan_5_int_Q5', 'chan_5_int_Q99', 'chan_6_int_Q5', 'chan_6_int_Q99', 'chan_7_int_Q5', 'chan_7_int_Q99', 'chan_8_int_Q5', 'chan_8_int_Q99', 'chan_9_int_Q5', 'chan_9_int_Q99')                                  
    
    if mode == 'gray':
        for root, _, files in os.walk(src+'/gray'):
            for file in files:
                name = os.path.splitext(file)[0]
                extension = os.path.splitext(file)[1]
                if extension == '.tif': 
                    gray_png = os.path.join(src+'/gray',name+'.png')
                    gray_orig = os.path.join(src+'/gray',file)
                    gray_orig_new = os.path.join(src+'/gray_orig',file)
                    gray = cv2.imread(gray_orig, -1)
                    
                    gray = exposure.rescale_intensity(gray, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)
                    shutil.move(gray_orig, gray_orig_new)
                    cv2.imwrite(gray_png, gray.astype(bitdepth))
                    print(file, chan_2_int_Q5, chan_2_int_Q99, chan_3_int_Q5, chan_3_int_Q99, chan_4_int_Q5, chan_4_int_Q99)
                                          
    if mode == 'rgb':
        if nr_of_channels == 2:
            for root, _, files in os.walk(src+'/rgb'):
                for file in files:
                    name = os.path.splitext(file)[0]
                    extension = os.path.splitext(file)[1]
                    if extension == '.tif': 
                        rgb_png = os.path.join(src+'/rgb',name+'.png')
                        rgb_orig = os.path.join(src+'/rgb',file)
                        rgb_orig_new = os.path.join(src+'/rgb_orig',file)
                        img = cv2.imread(rgb_orig, -1)
                        b, g = cv2.split(img) 

                        b = exposure.rescale_intensity(b, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)
                        g = exposure.rescale_intensity(g, (chan_2_int_Q5, chan_2_int_Q99)).astype(bitdepth)

                        rgb = cv2.merge((b,g))
                        shutil.move(rgb_orig, rgb_orig_new)
                        cv2.imwrite(rgb_png, rgb.astype(bitdepth))
                        print(file, chan_1_int_Q5, chan_1_int_Q99, chan_2_int_Q5, chan_2_int_Q99)
        if nr_of_channels == 3:
                    for root, _, files in os.walk(src+'/rgb'):
                        for file in files:
                            name = os.path.splitext(file)[0]
                            extension = os.path.splitext(file)[1]
                            if extension == '.tif': 
                                rgb_png = os.path.join(src+'/rgb',name+'.png')
                                rgb_orig = os.path.join(src+'/rgb',file)
                                rgb_orig_new = os.path.join(src+'/rgb_orig',file)
                                img = cv2.imread(rgb_orig, -1)
                                b, g, r = cv2.split(img) 

                                b = exposure.rescale_intensity(b, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)
                                g = exposure.rescale_intensity(g, (chan_2_int_Q5, chan_2_int_Q99)).astype(bitdepth)
                                r = exposure.rescale_intensity(r, (chan_3_int_Q5, chan_3_int_Q99)).astype(bitdepth)

                                rgb = cv2.merge((b,g,r))
                                shutil.move(rgb_orig, rgb_orig_new)
                                cv2.imwrite(rgb_png, rgb.astype(bitdepth))
                                print(file, chan_1_int_Q5, chan_1_int_Q99, chan_2_int_Q5, chan_2_int_Q99, chan_3_int_Q5, chan_3_int_Q99)
    
    if mode == 'rgba':
        for root, _, files in os.walk(src+'/rgba'):
            for file in files:
                name = os.path.splitext(file)[0]
                extension = os.path.splitext(file)[1]
                if extension == '.tif':
                    rgba_png = os.path.join(src+'/rgba',name+'.png')
                    rgba_orig = os.path.join(src+'/rgba',file)
                    rgba_orig_new = os.path.join(src+'/rgba_orig',file)
                    img = cv2.imread(rgba_orig, -1)
                    b, g, r, a = cv2.split(img)

                    b = exposure.rescale_intensity(b, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)
                    g = exposure.rescale_intensity(g, (chan_2_int_Q5, chan_2_int_Q99)).astype(bitdepth)
                    r = exposure.rescale_intensity(r, (chan_3_int_Q5, chan_3_int_Q99)).astype(bitdepth)
                    a = exposure.rescale_intensity(a, (chan_4_int_Q5, chan_4_int_Q99)).astype(bitdepth)

                    rgba = cv2.merge((b,g,r,a))
                    shutil.move(rgba_orig, rgba_orig_new)
                    cv2.imwrite(rgba_png, rgba.astype(bitdepth))
                    print(file, chan_1_int_Q5, chan_1_int_Q99, chan_2_int_Q5, chan_2_int_Q99, chan_3_int_Q5, chan_3_int_Q99, chan_4_int_Q5, chan_4_int_Q99)

    if mode == 'stack':
        if nr_of_channels == 1:
            for root, _, files in os.walk(src+'/stack'):
                for file in files:
                    name = os.path.splitext(file)[0]
                    extension = os.path.splitext(file)[1]
                    if extension == '.npy':
                        stack_orig = os.path.join(src+'/stack',file)
                        stack_orig_new = os.path.join(src+'/stack_orig',file)
                        array = np.load(stack_orig)

                        array = exposure.rescale_intensity(array, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)               

                        shutil.move(stack_orig, stack_orig_new)
                        np.save(stack_orig, stack.astype(bitdepth))
                        print(file, chan_1_int_Q5, chan_1_int_Q99)
        if nr_of_channels == 2:
            for root, _, files in os.walk(src+'/stack'):
                for file in files:
                    name = os.path.splitext(file)[0]
                    extension = os.path.splitext(file)[1]
                    if extension == '.npy':
                        stack_orig = os.path.join(src+'/stack',file)
                        stack_orig_new = os.path.join(src+'/stack_orig',file)
                        array = np.load(stack_orig)

                        a, b = np.split(img, 2)
                        a = exposure.rescale_intensity(a, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)
                        b = exposure.rescale_intensity(b, (chan_2_int_Q5, chan_2_int_Q99)).astype(bitdepth)
              
                        stack = np.stack([a, b], axis=-1)
                        shutil.move(stack_orig, stack_orig_new)
                        np.save(stack_orig, stack.astype(bitdepth))
                        print(file, chan_1_int_Q5, chan_1_int_Q99)
        if nr_of_channels == 3:
            for root, _, files in os.walk(src+'/stack'):
                for file in files:
                    name = os.path.splitext(file)[0]
                    extension = os.path.splitext(file)[1]
                    if extension == '.npy':
                        stack_png = os.path.join(src+'/stack',name+'.png')
                        stack_orig = os.path.join(src+'/stack',file)
                        stack_orig_new = os.path.join(src+'/stack_orig',file)
                        array = np.load(stack_orig)
                        
                        a, b, c = np.split(img, 3)
                        a = exposure.rescale_intensity(a, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)
                        b = exposure.rescale_intensity(b, (chan_2_int_Q5, chan_2_int_Q99)).astype(bitdepth)
                        c = exposure.rescale_intensity(c, (chan_3_int_Q5, chan_3_int_Q99)).astype(bitdepth)
              
                        stack = np.stack([a, b, c], axis=-1)
                        shutil.move(stack_orig, stack_orig_new)
                        np.save(stack_orig, stack.astype(bitdepth))
                        print(file, chan_1_int_Q5, chan_1_int_Q99, chan_2_int_Q5, chan_2_int_Q99, chan_3_int_Q5, chan_3_int_Q99)
        if nr_of_channels == 4:
            for root, _, files in os.walk(src+'/stack'):
                for file in files:
                    name = os.path.splitext(file)[0]
                    extension = os.path.splitext(file)[1]
                    if extension == '.npy':
                        stack_orig = os.path.join(src+'/stack',file)
                        stack_orig_new = os.path.join(src+'/stack_orig',file)
                        array = np.load(stack_orig, allow_pickle=True)
                        print(array.shape)
                        array_index=array.shape[2]
                        print('Array index: '+str(array_index))

                        chan_1, chan_2, chan_3, chan_4 = np.split(array, array_index, axis=2)   
                        
                        chan_1 = exposure.rescale_intensity(chan_1, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)
#chan_1_norm = normalize_array(chan_1, 1)
                        chan_2 = exposure.rescale_intensity(chan_2, (chan_2_int_Q5, chan_2_int_Q99)).astype(bitdepth)
#chan_2_norm = normalize_array(chan_2, 2)
                        chan_3 = exposure.rescale_intensity(chan_3, (chan_3_int_Q5, chan_3_int_Q99)).astype(bitdepth)
#chan_3_norm = normalize_array(chan_3, 3)
                        chan_4 = exposure.rescale_intensity(chan_4, (chan_4_int_Q5, chan_4_int_Q99)).astype(bitdepth)
#chan_4_norm = normalize_array(chan_4, 4)
#print(chan_4_norm.shape)
#stack = np.stack([chan_1_norm, chan_2_norm, chan_3_norm, chan_4_norm], axis=2)
                        stack = np.stack([chan_1, chan_2, chan_3, chan_4], axis=2)
                        stack = np.squeeze(stack, 3)
                        print(stack.shape)
                        shutil.move(stack_orig, stack_orig_new)
                        np.save(stack_orig, stack.astype(bitdepth))
                        print(file, chan_1_int_Q5, chan_1_int_Q99, chan_2_int_Q5, chan_2_int_Q99, chan_3_int_Q5, chan_3_int_Q99, chan_4_int_Q5, chan_4_int_Q99)
        if nr_of_channels == 5:
            for root, _, files in os.walk(src+'/stack'):
                for file in files:
                    name = os.path.splitext(file)[0]
                    extension = os.path.splitext(file)[1]
                    if extension == '.npy':
                        stack_orig = os.path.join(src+'/stack',file)
                        stack_orig_new = os.path.join(src+'/stack_orig',file)
                        array = np.load(stack_orig)
                        
                        a, b, c, d, e = np.split(img, 5)
                        a = exposure.rescale_intensity(a, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)
                        b = exposure.rescale_intensity(b, (chan_2_int_Q5, chan_2_int_Q99)).astype(bitdepth)
                        c = exposure.rescale_intensity(c, (chan_3_int_Q5, chan_3_int_Q99)).astype(bitdepth)
                        d = exposure.rescale_intensity(d, (chan_4_int_Q5, chan_4_int_Q99)).astype(bitdepth)
                        e = exposure.rescale_intensity(e, (chan_5_int_Q5, chan_5_int_Q99)).astype(bitdepth)
              
                        stack = np.stack([a, b, c, d, e], axis=-1)
                        shutil.move(stack_orig, stack_orig_new)
                        np.save(stack_orig, stack.astype(bitdepth))
                        print(file, chan_1_int_Q5, chan_1_int_Q99, chan_2_int_Q5, chan_2_int_Q99, chan_3_int_Q5, chan_3_int_Q99, chan_4_int_Q5, chan_4_int_Q99, chan_5_int_Q5, chan_5_int_Q99)
        if nr_of_channels == 6:
            for root, _, files in os.walk(src+'/stack'):
                for file in files:
                    name = os.path.splitext(file)[0]
                    extension = os.path.splitext(file)[1]
                    if extension == '.npy':
                        stack_orig = os.path.join(src+'/stack',file)
                        stack_orig_new = os.path.join(src+'/stack_orig',file)
                        array = np.load(stack_orig)
                        
                        a, b, c, d, e, f = np.split(img, 6)
                        a = exposure.rescale_intensity(a, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)
                        b = exposure.rescale_intensity(b, (chan_2_int_Q5, chan_2_int_Q99)).astype(bitdepth)
                        c = exposure.rescale_intensity(c, (chan_3_int_Q5, chan_3_int_Q99)).astype(bitdepth)
                        d = exposure.rescale_intensity(d, (chan_4_int_Q5, chan_4_int_Q99)).astype(bitdepth)
                        e = exposure.rescale_intensity(e, (chan_5_int_Q5, chan_5_int_Q99)).astype(bitdepth)
                        f = exposure.rescale_intensity(f, (chan_6_int_Q5, chan_6_int_Q99)).astype(bitdepth)

                        stack = np.stack([a, b, c, d, e, f], axis=-1)
                        shutil.move(stack_orig, stack_orig_new)
                        np.save(stack_orig, stack.astype(bitdepth))
                        print(file, chan_1_int_Q5, chan_1_int_Q99, chan_2_int_Q5, chan_2_int_Q99, chan_3_int_Q5, chan_3_int_Q99, chan_4_int_Q5, chan_4_int_Q99, chan_5_int_Q5, chan_5_int_Q99, chan_6_int_Q5, chan_6_int_Q99)
        if nr_of_channels == 7:
            for root, _, files in os.walk(src+'/stack'):
                for file in files:
                    name = os.path.splitext(file)[0]
                    extension = os.path.splitext(file)[1]
                    if extension == '.npy':
                        stack_orig = os.path.join(src+'/stack',file)
                        stack_orig_new = os.path.join(src+'/stack_orig',file)
                        array = np.load(stack_orig)
                        
                        a, b, c, d, e, f, g = np.split(img, 7)
                        a = exposure.rescale_intensity(a, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)
                        b = exposure.rescale_intensity(b, (chan_2_int_Q5, chan_2_int_Q99)).astype(bitdepth)
                        c = exposure.rescale_intensity(c, (chan_3_int_Q5, chan_3_int_Q99)).astype(bitdepth)
                        d = exposure.rescale_intensity(d, (chan_4_int_Q5, chan_4_int_Q99)).astype(bitdepth)
                        e = exposure.rescale_intensity(e, (chan_5_int_Q5, chan_5_int_Q99)).astype(bitdepth)
                        f = exposure.rescale_intensity(f, (chan_6_int_Q5, chan_6_int_Q99)).astype(bitdepth)
                        g = exposure.rescale_intensity(g, (chan_7_int_Q5, chan_7_int_Q99)).astype(bitdepth)
              
                        stack = np.stack([a, b, c, d, e, f, g], axis=-1)
                        shutil.move(stack_orig, stack_orig_new)
                        np.save(stack_orig, stack.astype(bitdepth))
                        print(file, chan_1_int_Q5, chan_1_int_Q99, chan_2_int_Q5, chan_2_int_Q99, chan_3_int_Q5, chan_3_int_Q99, chan_4_int_Q5, chan_4_int_Q99, chan_5_int_Q5, chan_5_int_Q99, chan_6_int_Q5, chan_6_int_Q99, chan_7_int_Q5, chan_7_int_Q99)
        if nr_of_channels == 8:
            for root, _, files in os.walk(src+'/stack'):
                for file in files:
                    name = os.path.splitext(file)[0]
                    extension = os.path.splitext(file)[1]
                    if extension == '.npy':
                        stack_orig = os.path.join(src+'/stack',file)
                        stack_orig_new = os.path.join(src+'/stack_orig',file)
                        array = np.load(stack_orig)
                        
                        a, b, c, d, e, f, g, h = np.split(img, 8)
                        a = exposure.rescale_intensity(a, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)
                        b = exposure.rescale_intensity(b, (chan_2_int_Q5, chan_2_int_Q99)).astype(bitdepth)
                        c = exposure.rescale_intensity(c, (chan_3_int_Q5, chan_3_int_Q99)).astype(bitdepth)
                        d = exposure.rescale_intensity(d, (chan_4_int_Q5, chan_4_int_Q99)).astype(bitdepth)
                        e = exposure.rescale_intensity(e, (chan_5_int_Q5, chan_5_int_Q99)).astype(bitdepth)
                        f = exposure.rescale_intensity(f, (chan_6_int_Q5, chan_6_int_Q99)).astype(bitdepth)
                        g = exposure.rescale_intensity(g, (chan_7_int_Q5, chan_7_int_Q99)).astype(bitdepth)
                        h = exposure.rescale_intensity(h, (chan_8_int_Q5, chan_8_int_Q99)).astype(bitdepth)
              
                        stack = np.stack([a, b, c, d, e, f, g, h], axis=-1)
                        shutil.move(stack_orig, stack_orig_new)
                        np.save(stack_orig, stack.astype(bitdepth))
                        print(file, chan_1_int_Q5, chan_1_int_Q99, chan_2_int_Q5, chan_2_int_Q99, chan_3_int_Q5, chan_3_int_Q99, chan_4_int_Q5, chan_4_int_Q99, chan_5_int_Q5, chan_5_int_Q99, chan_6_int_Q5, chan_6_int_Q99, chan_7_int_Q5, chan_7_int_Q99, chan_8_int_Q5, chan_8_int_Q99)
        if nr_of_channels == 9:
            for root, _, files in os.walk(src+'/stack'):
                for file in files:
                    name = os.path.splitext(file)[0]
                    extension = os.path.splitext(file)[1]
                    if extension == '.npy':
                        stack_orig = os.path.join(src+'/stack',file)
                        stack_orig_new = os.path.join(src+'/stack_orig',file)
                        array = np.load(stack_orig)
                        
                        a, b, c, d, e, f, g, h, i = np.split(img, 9)
                        a = exposure.rescale_intensity(a, (chan_1_int_Q5, chan_1_int_Q99)).astype(bitdepth)
                        b = exposure.rescale_intensity(b, (chan_2_int_Q5, chan_2_int_Q99)).astype(bitdepth)
                        c = exposure.rescale_intensity(c, (chan_3_int_Q5, chan_3_int_Q99)).astype(bitdepth)
                        d = exposure.rescale_intensity(d, (chan_4_int_Q5, chan_4_int_Q99)).astype(bitdepth)
                        e = exposure.rescale_intensity(e, (chan_5_int_Q5, chan_5_int_Q99)).astype(bitdepth)
                        f = exposure.rescale_intensity(f, (chan_6_int_Q5, chan_6_int_Q99)).astype(bitdepth)
                        g = exposure.rescale_intensity(g, (chan_7_int_Q5, chan_7_int_Q99)).astype(bitdepth)
                        h = exposure.rescale_intensity(h, (chan_8_int_Q5, chan_8_int_Q99)).astype(bitdepth)
                        i = exposure.rescale_intensity(i, (chan_9_int_Q5, chan_9_int_Q99)).astype(bitdepth)
              
                        stack = np.stack([a, b, c, d, e, f, g, h, i], axis=-1)
                        shutil.move(stack_orig, stack_orig_new)
                        np.save(stack_orig, stack.astype(bitdepth))
                        print(file, chan_1_int_Q5, chan_1_int_Q99, chan_2_int_Q5, chan_2_int_Q99, chan_3_int_Q5, chan_3_int_Q99, chan_4_int_Q5, chan_4_int_Q99, chan_5_int_Q5, chan_5_int_Q99, chan_6_int_Q5, chan_6_int_Q99, chan_7_int_Q5, chan_7_int_Q99, chan_8_int_Q5, chan_8_int_Q99, chan_9_int_Q5, chan_9_int_Q99)              

def plot_old_array(src, mode):
    imgs = []
    src = src+'/'+mode+'_orig'
    for path, subfolders, files in os.walk(src):
        for file in files:
            path = os.path.join(src,file)
            ext = os.path.splitext(path)[1]
            if ext == '.tif' or ext == '.png':
                print(file, 'im an image')
                imgs.append(path)
            if ext == '.npy':
                print(file, 'im an array')
                imgs.append(path)
    nimg = len(imgs)
    random_idx = random.choice(range(len(imgs)))        
    file=imgs[random_idx]        
    
    path = os.path.join(src,file)
    file_name = os.path.splitext(path)[0]
    ext = os.path.splitext(path)[1]

    if ext == '.tif' or ext == '.png':
        x = cv2.imread(file, -1) # img for cellpose
        x = cv2.cvtColor(file, cv2.COLOR_BGR2RGB)
        n_dim=len(x.shape) #shape of image
        dim=x.shape #dimensions of image
        channel=min(dim) #channel will be dimension with min value usually
        channel_position=dim.index(channel)
    elif ext == '.npy':
        x = np.load(path)
        n_dim=len(x.shape) #shape of image
        dim=x.shape #dimensions of image
        channel=min(dim) #channel will be dimension with min value usually
        channel_position=dim.index(channel)
    print('Number of images: ', nimg)
    print('Number of dimensions: ', n_dim)
    print('Channel index: ', channel_position)
    print('Image dimensions: ', dim)

    if n_dim > 3:
        channel_image=x.shape[2]
        fig, axs = plt.subplots(1, channel_image,figsize=(50,50))
        print("Image: %s" %(file_name))
        for channel in range(channel_image):
            axs[channel].imshow(x[:,:,channel])
            axs[channel].set_title('Channel '+str(channel+1),size=18)
            axs[channel].axis('off')
        fig.tight_layout()
        print('NOTE: If mode = stack channel order is reversed in cellpose')
        print('NOTE: blue = low intensity, yellow = high intensity, red = saturated intensity')
    if n_dim==3:
        channel_image=x.shape[2]
        fig, axs = plt.subplots(1, channel_image,figsize=(50,50))
        print("Image: %s" %(file_name))
        for channel in range(channel_image):
            axs[channel].imshow(x[:,:,channel])
            axs[channel].set_title('Channel '+str(channel+1),size=18)
            axs[channel].axis('off')
        fig.tight_layout()
        print('NOTE: If mode = stack channel order is reversed')
        print('NOTE: blue = low intensity, yellow = high intensity, red = saturated intensity')
    elif n_dim==2:
        print("One Channel")
        plt.imshow(x)
    else:
        print("Channel number invalid or dimensions wrong. Image shape is: "+str(x.shape))

#Read images
def plot_array(src, mode):
    imgs = []
    src = src+'/'+mode
    for path, subfolders, files in os.walk(src):
        for file in files:
            path = os.path.join(src,file)
            ext = os.path.splitext(path)[1]
            if ext == '.tif' or ext == '.png':
                print(file, 'im an image')
                imgs.append(path)
            if ext == '.npy':
                print(file, 'im an array')
                imgs.append(path)
    nimg = len(imgs)
    random_idx = random.choice(range(len(imgs)))        
    file=imgs[random_idx]        
    
    path = os.path.join(src,file)
    file_name = os.path.splitext(path)[0]
    ext = os.path.splitext(path)[1]

    if ext == '.tif' or ext == '.png':
        x = cv2.imread(file, -1) # img for cellpose
        x = cv2.cvtColor(file, cv2.COLOR_BGR2RGB)
        n_dim=len(x.shape) #shape of image
        dim=x.shape #dimensions of image
        channel=min(dim) #channel will be dimension with min value usually
        channel_position=dim.index(channel)
    elif ext == '.npy':
        x = np.load(path)
        n_dim=len(x.shape) #shape of image
        dim=x.shape #dimensions of image
        channel=min(dim) #channel will be dimension with min value usually
        channel_position=dim.index(channel)
    print('Number of images: ', nimg)
    print('Number of dimensions: ', n_dim)
    print('Channel index: ', channel_position)
    print('Image dimensions: ', dim)

    if n_dim > 3:
        channel_image=x.shape[2]
        fig, axs = plt.subplots(1, channel_image,figsize=(50,50))
        print("Image: %s" %(file_name))
        for channel in range(channel_image):
            axs[channel].imshow(x[:,:,channel])
            axs[channel].set_title('Channel '+str(channel+1),size=18)
            axs[channel].axis('off')
        fig.tight_layout()
        print('NOTE: If mode = stack channel order is reversed in cellpose')
        print('NOTE: blue = low intensity, yellow = high intensity, red = saturated intensity')
    if n_dim==3:
        channel_image=x.shape[2]
        fig, axs = plt.subplots(1, channel_image,figsize=(50,50))
        print("Image: %s" %(file_name))
        for channel in range(channel_image):
            axs[channel].imshow(x[:,:,channel])
            axs[channel].set_title('Channel '+str(channel+1),size=18)
            axs[channel].axis('off')
        fig.tight_layout()
        print('NOTE: If mode = stack channel order is reversed')
        print('NOTE: blue = low intensity, yellow = high intensity, red = saturated intensity')
    elif n_dim==2:
        print("One Channel")
        plt.imshow(x)
    else:
        print("Channel number invalid or dimensions wrong. Image shape is: "+str(x.shape))

def create_test_folder(src, mode):
    if os.path.exists(src+'/test/'+mode) is False:  
        os.mkdir(src+'/test/')
        os.mkdir(src+'/test/'+mode)
    print('Test folder path: ', src+'/test')
    contents = os.listdir(src+'/'+mode) # get contents of folder
    random.shuffle(contents) # shuffle the contents list
    if len(contents) <= 1:
        split_point = 1
    if len(contents) <= 2:
        split_point = 2
    if len(contents) <= 3:
        split_point = 3
    if len(contents) <= 4:
        split_point = 4
    if len(contents) <= 5:
        split_point = 5
    if len(contents) <= 6:
        split_point = 6
    if len(contents) <= 7:
        split_point = 7
    if len(contents) <= 8:
        split_point = 8
    if len(contents) <= 9:
        split_point = 9
    if len(contents) > 9:
        split_point = 10
    for img in contents[: split_point]:
        img_path = os.path.join(src+'/'+mode, img)
        test_path = os.path.join(src+'/test/'+mode, img)
        if os.path.exists(test_path) is False:         
            shutil.copy(img_path, test_path)
        else:
            print(img, 'already in test folder')
            
            
def run_cellpose_segmentation(src, mode, number_of_objects, view_mask):
    
    if view_mask == 'on':
        print('WARNING: plotting masks slows down analasys')
    
    if mode != 'stack':
        src_mask = src+'/masks'
        if os.path.exists(src_mask) is False:         
            os.mkdir(src_mask)
    
    if mode == 'grey':
        ch1 = [0, 0]
        if obj1 == 'nuclei':
            model1 = models.Cellpose(gpu=True, model_type='nuclei')
            print('Nuclei model enabled '+'for '+obj1+' Segmentation from grayscale image')

        if obj1 != 'cells':
            model1 = models.Cellpose(gpu=True, model_type='cyto')
            print('Cytoplasmic model enabled '+'for '+obj1+' Segmentation from grayscale image')
    
    if mode == 'rgb':
        if obj1 == 'nuclei':
            ch1 = [obj1_channel, 0]
            model1 = models.Cellpose(gpu=True, model_type='nuclei')
            print('Nuclei model enabled '+'for '+obj1+' Segmentation '+' in  channel: '+str(ch1))
        if obj1 != 'cells':
            ch1 = [obj1_channel, 0]
            model1 = models.Cellpose(gpu=True, model_type='cyto')
            print('Cytoplasmic model enabled '+'for '+obj1+' Segmentation')

    if mode == 'rgba':
        if obj1 == 'nuclei':
            ch1 = [obj1_channel, 0]
            model1 = models.Cellpose(gpu=True, model_type='nuclei')
            print('Nuclei model enabled '+'for '+obj1+' Segmentation '+' in  channel: '+str(ch1))
        if obj1 != 'cells':
            ch1 = [obj1_channel, 0]
            model1 = models.Cellpose(gpu=True, model_type='cyto')
            print('Cytoplasmic model enabled '+'for '+obj1+' Segmentation')
             
    if mode == 'stack':
        if obj1 == 'nuclei':
            ch1 = [obj1_channel, 0]
            model1 = models.Cellpose(gpu=True, model_type='nuclei')
            print('Nuclei model enabled '+'for '+obj1+' Segmentation '+' in  channel: '+str(ch1))
        if obj1 != 'cells':
            ch1 = [obj1_channel, 0]
            model1 = models.Cellpose(gpu=True, model_type='cyto')
            print('Cytoplasmic model enabled '+'for '+obj1+' Segmentation')

    path = src
    src = src+'/'+mode    
    for name in os.listdir(src):
        file = os.path.join(src, name)
        if mode != 'stack':
            mask_name=name.replace('.png', '.tif')
            if os.path.exists(src_mask+"/"+mask_name) is True:
                print("mask exists: "+name)
        else:
            extension=os.path.splitext(name)[1]
            if extension == '.csv':
                print(name, 'is a csv file')
            else:
                if mode == 'gray' and extension == '.png':
                    image = cv2.imread(file, -1) # img for cellpose
                    image_show = img_as_ubyte(image) # img for matplotlib
                    print(image.shape)
                if mode == 'gray' and extension != '.png':
                    print(file+': is not png')
                if mode == 'rgb' and extension == '.png':
                    image = cv2.imread(file, -1) # img for cellpose
                    image_show = img_as_ubyte(image) # img for matplotlib
                    print(image.shape)
                if mode == 'rgb' and extension != '.png':
                    print(file+': is not png')
                if mode == 'rgba' and extension == '.png':
                    image = cv2.imread(file, -1) # img for cellpose
                    image_show = img_as_ubyte(image) # img for matplotlib
                    print(image.shape)
                if mode == 'rgba' and extension != '.png':
                    print(file+': is not png')
                    pass
                if mode == 'stack' and extension == '.npy':
                    array = np.load(file)
                    array_index=array.shape[2]
                    print('Array index: '+str(array_index))
                    if array_index == 1:
                        image = cv2.merge((array), array_index) # img for cellpose
                        image_show = img_as_ubyte(image) # img for matplotlib
                        image2 = image
                    if array_index == 2:
                        chan_1, chan_2 = np.split(array, array_index, axis=2)
                        image = cv2.merge((chan_1, chan_2), array_index) 
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # img for cellpose
                        image_show = img_as_ubyte(image) # img for matplotlib
                        image2 = image
                    if array_index == 3:
                        chan_1, chan_2, chan_3 = np.split(array, array_index, axis=2)
                        image = cv2.merge((chan_1, chan_2, chan_3), array_index) 
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # img for cellpose
                        image_show = img_as_ubyte(image) # img for matplotlib
                        image2 = image
                    if array_index == 4:
                        chan_1, chan_2, chan_3, chan_4 = np.split(array, array_index, axis=2)
                        image = cv2.merge((chan_1, chan_2, chan_3), 3) 
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # img for cellpose
                        image_show = img_as_ubyte(image) # img for matplotlib
                        image2 = image
                    if array_index == 5:
                        chan_1, chan_2, chan_3, chan_4, chan_5 = np.split(array, array_index, axis=2)
                        image = cv2.merge(merg_channels_index, 3) 
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # img for cellpose
                        image_show = img_as_ubyte(image) # img for matplotlib
                        image2 = image
                    if array_index == 6:
                        chan_1, chan_2, chan_3, chan_4, chan_5, chan_6 = np.split(array, array_index, axis=2)
                        image = cv2.merge(merg_channels_index, 3) 
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # img for cellpose
                        image_show = img_as_ubyte(image) # img for matplotlib
                        image2 = image
                    if array_index == 7:
                        chan_1, chan_2, chan_3, chan_4, chan_5, chan_6, chan_7 = np.split(array, array_index, axis=2)
                        image = cv2.merge(merg_channels_index, 3) 
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # img for cellpose
                        image_show = img_as_ubyte(image) # img for matplotlib
                        image2 = image
                    if array_index == 8:
                        chan_1, chan_2, chan_3, chan_4, chan_5, chan_6, chan_7, chan_8 = np.split(array, array_index, axis=2)
                        image = cv2.merge(merg_channels_index, 3) 
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # img for cellpose
                        image_show = img_as_ubyte(image) # img for matplotlib
                        image2 = image
                    if array_index == 9:
                        chan_1, chan_2, chan_3, chan_4, chan_5, chan_6, chan_7, chan_8, chan_9 = np.split(array, array_index, axis=2)
                        image = cv2.merge(merg_channels_index, 3) 
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # img for cellpose
                        image_show = img_as_ubyte(image) # img for matplotlib
                        image2 = image

                    os.chdir(src)
                    try:
                        #find primary object
                        if gausian1 != 0:
                            kernel = np.ones((gausian1, gausian1),np.float32)/(gausian1*gausian1)
                            image = cv2.filter2D(image,-1, kernel)
                            image_show = img_as_ubyte(image)
                        short_name = os.path.splitext(name)
                        min1 = (dim1*dim1)/2
                        print(obj1+' minimum Nr. of px: '+str(min1)+' Mean diamiter (px): '+str(dim1))
                        masks, flows, styles, diams = model1.eval(image,
                                                                  batch_size = batch_size, # (int (optional, default 8)) – number of 224x224 patches to run simultaneously on the GPU (can make smaller or bigger depending on GPU memory usage)
                                                                  channels = ch1, # (list (optional, default None)) – list of channels, either of length 2 or of length number of images by 2. First element of list is the channel to segment (0=grayscale, 1=red, 2=green, 3=chan_4). Second element of list is the optional nuclear channel (0=none, 1=red, 2=green, 3=chan_4). For instance, to segment grayscale images, input [0,0]. To segment images with cells in green and nuclei in blue, input [2,3]. To segment one grayscale image and one image with cells in green and nuclei in blue, input [[0,0], [2,3]].
                                                                  channel_axis = None, # (int (optional, default None)) – if None, channels dimension is attempted to be automatically determined
                                                                  z_axis = None, # (int (optional, default None)) – if None, z dimension is attempted to be automatically determined
                                                                  normalize = True, # (bool (default, True)) – normalize data so 0.0=1st percentile and 1.0=99th percentile of image intensities in each channel
                                                                  invert = False, # (bool (optional, default False)) – invert image pixel intensity before running network
                                                                  rescale = True, # (float (optional, default None)) – resize factor for each image, if None, set to 1.0
                                                                  diameter = dim1, # (float (optional, default None)) – diameter for each image (only used if rescale is None), if diameter is None, set to diam_mean
                                                                  do_3D = False, # (bool (optional, default False)) – set to True to run 3D segmentation on 4D image input
                                                                  anisotropy = None, # (float (optional, default None)) – for 3D segmentation, optional rescaling factor (e.g. set to 2.0 if Z is sampled half as dense as X or Y)
                                                                  net_avg = True, # (bool (optional, default True)) – runs the 4 built-in networks and averages them if True, runs one network if False
                                                                  augment = True, # (bool (optional, default False)) – tiles image with overlapping tiles and flips overlapped regions to augment
                                                                  tile = True, # (bool (optional, default True)) – tiles image to ensure GPU/CPU memory usage limited (recommended)
                                                                  tile_overlap = 0.1, # (float (optional, default 0.1)) – fraction of overlap of tiles when computing flows
                                                                  resample = True, # (bool (optional, default False)) – run dynamics at original image size (will be slower but create more accurate boundaries)
                                                                  interp = True, # (bool (optional, default True)) – interpolate during 2D dynamics (not available in 3D) (in previous versions it was False)
                                                                  flow_threshold = flow_threshold, # (float (optional, default 0.4)) – flow error threshold (all cells with errors below threshold are kept) (not used for 3D)
                                                                  mask_threshold = mt1, # (float (optional, default 0.0)) – all pixels with value above threshold kept for masks, decrease to find more and larger masks
                                                                  # DEPRECEATED: dist_threshold = None, # (float (optional, default None) DEPRECATED) – use mask_threshold instead
                                                                  # DEPRECEATED: cellprob_threshold = primary_ct, # (float (optional, default None) DEPRECATED) – use mask_threshold instead
                                                                  #compute_masks = False, # (bool (optional, default True)) – Whether or not to compute dynamics and return masks. This is set to False when retrieving the styles for the size model.
                                                                  min_size = min1, # (int (optional, default 15)) – minimum number of pixels per mask, can turn off with -1
                                                                  stitch_threshold = 0.0, # (float (optional, default 0.0)) – if stitch_threshold>0.0 and not do_3D, masks are stitched in 3D to return volume segmentation
                                                                  progress = False, # (pyqt progress bar (optional, default None)) – to return progress bar status to GUI
                                                                  omni = True, # (bool (optional, default False)) – use omnipose mask recontruction features
                                                                  #calc_trace = False, # (bool (optional, default False)) – calculate pixel traces and return as part of the flow
                                                                  verbose = True, # (bool (optional, default False)) – turn on additional output to logs for debugging
                                                                  transparency = False) #(bool (optional, default False)) – modulate flow opacity by magnitude instead of brightness (can use flows on any color background) diameter=primary_dim
                        obj1_mask = masks
                        print(obj1_mask.shape)
                        if view_mask == 'on':
                            print(obj1)
                            maski = masks
                            flowi = flows[0]
                            fig = plt.figure(figsize=(50,50))
                            plot.show_segmentation(fig, image_show, maski, flowi, channels=ch1)
                            plt.tight_layout()
                            plt.show()

                        if gausian1 != 0:
                            image = image2

                    except:
                        print('WARNING: Segmentation of '+obj1+' for image: '+name+' FAILED')
                    try:
                        if obj2 != 'off':
                            if obj1 == 'nuclei':
                                if obj2_model != 'cyto':
                                    ch2 = [obj2_channel, obj1_channel]
                                    model2 = models.Cellpose(gpu=True, model_type="cyto2")
                                    print('Cytoplasmic model with nuclei enabled '+'for '+obj2+' Segmentation')
                                if obj2_model == 'cyto':
                                    ch2 = [obj2_channel, 0]
                                    model2 = models.Cellpose(gpu=True, model_type="cyto")
                                    print('Cytoplasmic model '+'for '+obj2+' Segmentation')
                        if obj1 != 'nuclei':
                            ch2 = [obj2_channel, 0]
                            model2 = models.Cellpose(gpu=True, model_type="cyto")
                            print('Cytoplasmic model enabled '+'for '+obj2+' Segmentation')

                        #find secondary object
                        if gausian2 != 0:
                            kernel = np.ones((gausian2, gausian2),np.float32)/(gausian2*gausian2)
                            image = cv2.filter2D(image,-1, kernel)
                            image_show = img_as_ubyte(image)

                        min2 = (dim2*dim2)/2
                        print(obj2+' minimum Nr. of px: '+str(min2)+' Mean diamiter (px): '+str(dim2))
                        masks, flows, styles, diams = model2.eval(image,
                                                                  batch_size = batch_size, # (int (optional, default 8)) – number of 224x224 patches to run simultaneously on the GPU (can make smaller or bigger depending on GPU memory usage)
                                                                  channels = ch2, # (list (optional, default None)) – list of channels, either of length 2 or of length number of images by 2. First element of list is the channel to segment (0=grayscale, 1=red, 2=green, 3=blue). Second element of list is the optional nuclear channel (0=none, 1=red, 2=green, 3=blue). For instance, to segment grayscale images, input [0,0]. To segment images with cells in green and nuclei in blue, input [2,3]. To segment one grayscale image and one image with cells in green and nuclei in blue, input [[0,0], [2,3]].
                                                                  channel_axis = None, # (int (optional, default None)) – if None, channels dimension is attempted to be automatically determined
                                                                  z_axis = None, # (int (optional, default None)) – if None, z dimension is attempted to be automatically determined
                                                                  normalize = True, # (bool (default, True)) – normalize data so 0.0=1st percentile and 1.0=99th percentile of image intensities in each channel
                                                                  invert = False, # (bool (optional, default False)) – invert image pixel intensity before running network
                                                                  rescale = True, # (float (optional, default None)) – resize factor for each image, if None, set to 1.0
                                                                  diameter = dim2, # (float (optional, default None)) – diameter for each image (only used if rescale is None), if diameter is None, set to diam_mean
                                                                  do_3D = False, # (bool (optional, default False)) – set to True to run 3D segmentation on 4D image input
                                                                  anisotropy = None, # (float (optional, default None)) – for 3D segmentation, optional rescaling factor (e.g. set to 2.0 if Z is sampled half as dense as X or Y)
                                                                  net_avg = True, # (bool (optional, default True)) – runs the 4 built-in networks and averages them if True, runs one network if False
                                                                  augment = True, # (bool (optional, default False)) – tiles image with overlapping tiles and flips overlapped regions to augment
                                                                  tile = True, # (bool (optional, default True)) – tiles image to ensure GPU/CPU memory usage limited (recommended)
                                                                  tile_overlap = 0.1, # (float (optional, default 0.1)) – fraction of overlap of tiles when computing flows
                                                                  resample = True, # (bool (optional, default False)) – run dynamics at original image size (will be slower but create more accurate boundaries)
                                                                  interp = True, # (bool (optional, default True)) – interpolate during 2D dynamics (not available in 3D) (in previous versions it was False)
                                                                  flow_threshold = flow_threshold, # (float (optional, default 0.4)) – flow error threshold (all cells with errors below threshold are kept) (not used for 3D)
                                                                  mask_threshold = mt2, # (float (optional, default 0.0)) – all pixels with value above threshold kept for masks, decrease to find more and larger masks
                                                                  # DEPRECEATED: dist_threshold = None, # (float (optional, default None) DEPRECATED) – use mask_threshold instead
                                                                  # DEPRECEATED: cellprob_threshold = secondary_ct, # (float (optional, default None) DEPRECATED) – use mask_threshold instead
                                                                  #compute_masks = True, # (bool (optional, default True)) – Whether or not to compute dynamics and return masks. This is set to False when retrieving the styles for the size model.
                                                                  min_size = min2, # (int (optional, default 15)) – minimum number of pixels per mask, can turn off with -1
                                                                  stitch_threshold = 0.0, # (float (optional, default 0.0)) – if stitch_threshold>0.0 and not do_3D, masks are stitched in 3D to return volume segmentation
                                                                  progress = True, # (pyqt progress bar (optional, default None)) – to return progress bar status to GUI
                                                                  omni = True, # (bool (optional, default False)) – use omnipose mask recontruction features
                                                                  #calc_trace = False, # (bool (optional, default False)) – calculate pixel traces and return as part of the flow
                                                                  verbose = True, # (bool (optional, default False)) – turn on additional output to logs for debugging
                                                                  transparency = False) #(bool (optional, default False)) – modulate flow opacity by magnitude instead of brightness (can use flows on any color background)

                        obj2_mask = masks
                        print(obj2_mask.shape)
                        if view_mask == 'on':
                            print(obj2)
                            maski = masks
                            flowi = flows[0]
                            fig = plt.figure(figsize=(50,50))
                            plot.show_segmentation(fig, image_show, obj2_mask, flowi, channels=ch2)
                            plt.tight_layout()
                            plt.show() #plt.subplot(122),plt.imshow(dst),plt.title('Averaging')

                        if gausian2 != 0:
                            image = image2

                    except:
                        print('WARNING: Segmentation of '+obj2+' '+' for image: '+'name '+'FAILED')
                    try:
                        if obj3 != 'off':
                            if obj3_model == 'cyto':
                                ch3 = [obj3_channel, 0]
                                model3 = models.Cellpose(gpu=True, model_type="cyto")
                                print('Cytoplasmic model '+'for '+obj3+' Segmentation')
                            if obj3_model == 'cyto2':
                                ch3 = [obj3_channel, obj1_channel]
                                model3 = models.Cellpose(gpu=True, model_type="cyto2")
                                print('Cytoplasmic model with nuclei '+'for '+obj3+' Segmentation')
                            if obj3_model == 'nuceli':
                                ch3 = [obj3_channel, 0]
                                model3 = models.Cellpose(gpu=True, model_type="nuclei")
                                print('Nuclei model '+'for '+obj3+' Segmentation')

                        #find secondary object
                        if gausian3 != 0:
                            kernel = np.ones((gausian3, gausian3),np.float32)/(gausian3*gausian3)
                            image = cv2.filter2D(image,-1, kernel)
                            image_show = img_as_ubyte(image)
                        min3 = (dim3*dim3)/2
                        print(obj3+' minimum Nr. of px: '+str(min3)+' Mean diamiter (px): '+str(dim3))
                        masks, flows, styles, diams = model3.eval(image,
                                                                  batch_size = batch_size, # (int (optional, default 8)) – number of 224x224 patches to run simultaneously on the GPU (can make smaller or bigger depending on GPU memory usage)
                                                                  channels = ch3, # (list (optional, default None)) – list of channels, either of length 2 or of length number of images by 2. First element of list is the channel to segment (0=grayscale, 1=red, 2=green, 3=blue). Second element of list is the optional nuclear channel (0=none, 1=red, 2=green, 3=blue). For instance, to segment grayscale images, input [0,0]. To segment images with cells in green and nuclei in blue, input [2,3]. To segment one grayscale image and one image with cells in green and nuclei in blue, input [[0,0], [2,3]].
                                                                  channel_axis = None, # (int (optional, default None)) – if None, channels dimension is attempted to be automatically determined
                                                                  z_axis = None, # (int (optional, default None)) – if None, z dimension is attempted to be automatically determined
                                                                  normalize = True, # (bool (default, True)) – normalize data so 0.0=1st percentile and 1.0=99th percentile of image intensities in each channel
                                                                  invert = False, # (bool (optional, default False)) – invert image pixel intensity before running network
                                                                  rescale = True, # (float (optional, default None)) – resize factor for each image, if None, set to 1.0
                                                                  diameter = dim3, # (float (optional, default None)) – diameter for each image (only used if rescale is None), if diameter is None, set to diam_mean
                                                                  do_3D = False, # (bool (optional, default False)) – set to True to run 3D segmentation on 4D image input
                                                                  anisotropy = None, # (float (optional, default None)) – for 3D segmentation, optional rescaling factor (e.g. set to 2.0 if Z is sampled half as dense as X or Y)
                                                                  net_avg = True, # (bool (optional, default True)) – runs the 4 built-in networks and averages them if True, runs one network if False
                                                                  augment = True, # (bool (optional, default False)) – tiles image with overlapping tiles and flips overlapped regions to augment
                                                                  tile = True, # (bool (optional, default True)) – tiles image to ensure GPU/CPU memory usage limited (recommended)
                                                                  tile_overlap = 0.1, # (float (optional, default 0.1)) – fraction of overlap of tiles when computing flows
                                                                  resample = True, # (bool (optional, default False)) – run dynamics at original image size (will be slower but create more accurate boundaries)
                                                                  interp = True, # (bool (optional, default True)) – interpolate during 2D dynamics (not available in 3D) (in previous versions it was False)
                                                                  flow_threshold = flow_threshold, # (float (optional, default 0.4)) – flow error threshold (all cells with errors below threshold are kept) (not used for 3D)
                                                                  mask_threshold = mt3, # (float (optional, default 0.0)) – all pixels with value above threshold kept for masks, decrease to find more and larger masks
                                                                  # DEPRECEATED: dist_threshold = None, # (float (optional, default None) DEPRECATED) – use mask_threshold instead
                                                                  # DEPRECEATED: cellprob_threshold = secondary_ct, # (float (optional, default None) DEPRECATED) – use mask_threshold instead
                                                                  #compute_masks = True, # (bool (optional, default True)) – Whether or not to compute dynamics and return masks. This is set to False when retrieving the styles for the size model.
                                                                  min_size = min3, # (int (optional, default 15)) – minimum number of pixels per mask, can turn off with -1
                                                                  stitch_threshold = 0.0, # (float (optional, default 0.0)) – if stitch_threshold>0.0 and not do_3D, masks are stitched in 3D to return volume segmentation
                                                                  progress = True, # (pyqt progress bar (optional, default None)) – to return progress bar status to GUI
                                                                  omni = True, # (bool (optional, default False)) – use omnipose mask recontruction features
                                                                  #calc_trace = False, # (bool (optional, default False)) – calculate pixel traces and return as part of the flow
                                                                  verbose = True, # (bool (optional, default False)) – turn on additional output to logs for debugging
                                                                  transparency = False) #(bool (optional, default False)) – modulate flow opacity by magnitude instead of brightness (can use flows on any color background)

                        obj3_mask = masks
                        print(obj3_mask.shape)
                        if view_mask == 'on':
                            print(obj3)
                            maski = masks
                            flowi = flows[0]
                            fig = plt.figure(figsize=(50,50))
                            plot.show_segmentation(fig, image_show, obj3_mask, flowi, channels=ch3)
                            plt.tight_layout()
                            plt.show() #plt.subplot(122),plt.imshow(dst),plt.title('Averaging')

                    except:
                        print('WARNING: Segmentation of '+obj3+' '+' for image: '+'name '+'FAILED')
                    
                    if os.path.basename(path) != 'test': 
                        if mode != 'stack':
                            if nr_of_channels >= 1:
                                if os.path.exists(src_mask+'/'+obj1) is False:
                                    os.mkdir(src_mask+'/'+obj1)
                                os.chdir(src_mask+'/'+obj1)
                                imsave(str(short_name[0])+".tif", obj1_mask, compress=ZIP_DEFLATED)
                                print("Saved: "+name+' '+obj1)
                            if nr_of_channels >= 2:
                                if os.path.exists(src_mask+'/'+obj2) is False:
                                    os.mkdir(src_mask+'/'+obj2)
                                os.chdir(src_mask+'/'+obj2)
                                imsave(str(short_name[0])+".tif", obj2_mask, compress=ZIP_DEFLATED)
                                print("Saved: "+name+' '+obj2)
                            if nr_of_channels >= 3:
                                if os.path.exists(src_mask+'/'+obj3) is False:
                                    os.mkdir(src_mask+'/'+obj3)
                                os.chdir(src_mask+'/'+obj3)
                                imsave(str(short_name[0])+".tif", obj3_mask, compress=ZIP_DEFLATED)
                                print("Saved: "+name+' '+obj3)

                    if mode == 'stack':
                        print('FILE', file)
                        if number_of_objects == 1:
                            if nr_of_channels == 1:
                                chan_1 = np.squeeze(chan_1, 2)
                                stack = np.stack([chan_1, obj1_mask], axis=2)

                            if nr_of_channels == 2:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                stack = np.stack([chan_1,chan_2, obj1_mask], axis=2)

                            if nr_of_channels == 3:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                stack = np.stack([chan_1, chan_2, chan_3, obj1_mask], axis=2)

                            if nr_of_channels == 4:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                stack = np.stack([chan_1, chan_2,chan_3,chan_4, obj1_mask], axis=2)

                            if nr_of_channels == 5:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                stack = np.stack([chan_1, chan_2, chan_3,chan_4, chan_5, obj1_mask], axis=2)

                            if nr_of_channels == 6:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                chan_6 = np.squeeze(chan_6, 2)
                                stack = np.stack([chan_1,chan_2,chan_3,chan_4, chan_5, chan_6, obj1_mask], axis=2)

                            if nr_of_channels == 7:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                chan_6 = np.squeeze(chan_6, 2)
                                chan_7 = np.squeeze(chan_7, 2)
                                stack = np.stack([chan_1,chan_2,chan_3,chan_4, chan_5, chan_6, chan_7, obj1_mask], axis=2)

                            if nr_of_channels == 8:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                chan_6 = np.squeeze(chan_6, 2)
                                chan_7 = np.squeeze(chan_7, 2)
                                chan_8 = np.squeeze(chan_8, 2)
                                stack = np.stack([chan_1, chan_2,chan_3,chan_4, chan_5, chan_6, chan_7, chan_8, obj1_mask], axis=2)

                            if nr_of_channels == 9:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                chan_6 = np.squeeze(chan_6, 2)
                                chan_7 = np.squeeze(chan_7, 2)
                                chan_8 = np.squeeze(chan_8, 2)
                                chan_9 = np.squeeze(chan_9, 2)
                                stack = np.stack([chan_1, chan_2, chan_3, chan_4, chan_5, chan_6, chan_7, chan_8, chan_9, obj1_mask], axis=2)
                            print(stack.shape)
                            print("Saved: "+name+' '+obj1)
                            np.save(file, stack.astype(bitdepth))

                        if number_of_objects == 2:
                            if nr_of_channels == 1:
                                chan_1 = np.squeeze(chan_1, 2)
                                stack = np.stack([chan_1, obj1_mask, obj2_mask], axis=2)

                            if nr_of_channels == 2:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                stack = np.stack([chan_1,chan_2, obj1_mask, obj2_mask], axis=2)

                            if nr_of_channels == 3:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                stack = np.stack([chan_1, chan_2, chan_3, obj1_mask, obj2_mask], axis=2)

                            if nr_of_channels == 4:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                stack = np.stack([chan_1, chan_2,chan_3,chan_4, obj1_mask, obj2_mask], axis=2)

                            if nr_of_channels == 5:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                stack = np.stack([chan_1, chan_2, chan_3,chan_4, chan_5, obj1_mask, obj2_mask], axis=2)

                            if nr_of_channels == 6:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                chan_6 = np.squeeze(chan_6, 2)
                                stack = np.stack([chan_1,chan_2,chan_3,chan_4, chan_5, chan_6, obj1_mask, obj2_mask], axis=2)

                            if nr_of_channels == 7:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                chan_6 = np.squeeze(chan_6, 2)
                                chan_7 = np.squeeze(chan_7, 2)
                                stack = np.stack([chan_1,chan_2,chan_3,chan_4, chan_5, chan_6, chan_7, obj1_mask, obj2_mask], axis=2)

                            if nr_of_channels == 8:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                chan_6 = np.squeeze(chan_6, 2)
                                chan_7 = np.squeeze(chan_7, 2)
                                chan_8 = np.squeeze(chan_8, 2)
                                stack = np.stack([chan_1, chan_2,chan_3,chan_4, chan_5, chan_6, chan_7, chan_8, obj1_mask, obj2_mask], axis=2)

                            if nr_of_channels == 9:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                chan_6 = np.squeeze(chan_6, 2)
                                chan_7 = np.squeeze(chan_7, 2)
                                chan_8 = np.squeeze(chan_8, 2)
                                chan_9 = np.squeeze(chan_9, 2)
                                stack = np.stack([chan_1, chan_2, chan_3, chan_4, chan_5, chan_6, chan_7, chan_8, chan_9, obj1_mask, obj2_mask], axis=2)
                            print(stack.shape)
                            print("Saved: "+name+' '+obj1+' '+obj2)
                            np.save(file, stack.astype(bitdepth))
                            
                        if number_of_objects == 3:
                            if nr_of_channels == 1:
                                chan_1 = np.squeeze(chan_1, 2)
                                stack = np.stack([chan_1, obj1_mask, obj2_mask, obj3_mask], axis=2)

                            if nr_of_channels == 2:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                stack = np.stack([chan_1,chan_2, obj1_mask, obj2_mask, obj3_mask], axis=2)

                            if nr_of_channels == 3:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                stack = np.stack([chan_1, chan_2, chan_3, obj1_mask, obj2_mask, obj3_mask], axis=2)

                            if nr_of_channels == 4:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                stack = np.stack([chan_1, chan_2, chan_3,chan_4, obj1_mask, obj2_mask, obj3_mask], axis=2)
                                
                            if nr_of_channels == 5:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                stack = np.stack([chan_1, chan_2, chan_3,chan_4, chan_5, obj1_mask, obj2_mask, obj3_mask], axis=2)

                            if nr_of_channels == 6:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                chan_6 = np.squeeze(chan_6, 2)
                                stack = np.stack([chan_1,chan_2,chan_3,chan_4, chan_5, chan_6, obj1_mask, obj2_mask, obj3_mask], axis=2)

                            if nr_of_channels == 7:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                chan_6 = np.squeeze(chan_6, 2)
                                chan_7 = np.squeeze(chan_7, 2)
                                stack = np.stack([chan_1,chan_2,chan_3,chan_4, chan_5, chan_6, chan_7, obj1_mask, obj2_mask, obj3_mask], axis=2)

                            if nr_of_channels == 8:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                chan_6 = np.squeeze(chan_6, 2)
                                chan_7 = np.squeeze(chan_7, 2)
                                chan_8 = np.squeeze(chan_8, 2)
                                stack = np.stack([chan_1, chan_2,chan_3,chan_4, chan_5, chan_6, chan_7, chan_8, obj1_mask, obj2_mask, obj3_mask], axis=2)

                            if nr_of_channels == 9:
                                chan_1 = np.squeeze(chan_1, 2)
                                chan_2 = np.squeeze(chan_2, 2)
                                chan_3 = np.squeeze(chan_3, 2)
                                chan_4 = np.squeeze(chan_4, 2)
                                chan_5 = np.squeeze(chan_5, 2)
                                chan_6 = np.squeeze(chan_6, 2)
                                chan_7 = np.squeeze(chan_7, 2)
                                chan_8 = np.squeeze(chan_8, 2)
                                chan_9 = np.squeeze(chan_9, 2)
                                stack = np.stack([chan_1, chan_2, chan_3, chan_4, chan_5, chan_6, chan_7, chan_8, chan_9, obj1_mask, obj2_mask, obj3_mask], axis=2)
                            print(stack.shape)
                            print("Saved: "+name+' '+obj1+' '+obj2+' '+obj3)
                            np.save(file, stack.astype(bitdepth))

In [ ]:
src = r'C:\Users\einar\Desktop\omni2'
nr_of_channels = 4

In [ ]:
move_to_chan_folder(src, nr_of_channels)

In [ ]:
bitdepth = np.uint16
mode = 'stack'

In [ ]:
gray_to_color(src, nr_of_channels, mode)

In [ ]:
mode = 'stack'
plot_array(src, mode)

In [ ]:
rescal_intensity(src, nr_of_channels, mode)

In [ ]:
#Select paramiters for cellpose segmentation

# Primary object segmentation, if nuclei are present set this object to nuclei
obj1 = 'nuclei' # Set an appropreate name
obj1_channel = 3 # Set the channel the object will be segmented from 
dim1 = 50 # Set diamiter in pixels (length in pixels)
mt1 = 0 # Threshold for cellpose, lower if objects are not detected, increase if to many objects are detected
gausian1 = 0

# Secondary object detection
obj2 = 'cells' # Set an appropreate name or set off
obj2_channel = 1 # Set the channel the object will be segmented from 
dim2 = 160 # Set diamiter in pixels (length in pixels)
mt2 = 0 # Threshold for cellpose, lower if object0s are not detected, increase if to many objects are detected
obj2_model = 'cyto2'
gausian2 = 2

#Tertery object detection
obj3 = 'toxoplasma' # Set an appropreate name or set off
obj3_channel = 2 # Set the channel the object will be segmented from 
dim3 = 30 # Set diamiter in pixels (length in pixels)
mt3 = 0 # Threshold for cellpose, lower if objects are not detected, increase if to many objects are detected
obj3_model = 'cyto' #set to yes if the tertery object is nucleated and you have a nuclear channel
mode = 'stack' # Set to gray, rgb, rgba or stack
gausian3 = 0

#Common paramiters
number_of_objects = 3
merg_channels_index = 'chan_1, chan_2, chan_3'
print(merg_channels_index)
batch_size = 24 #Default is 8, higher values use more GPU VRAM
flow_threshold = 10
print(src)

In [ ]:
#Run this cell to create test folder and test out different cellpose settings
#create_test_folder(src, mode)
#if os.path.basename(src) != 'test':
#    src = src+'/test'
    
#print(src)

In [ ]:
run_cellpose_segmentation(src, mode='stack', number_of_objects=3, view_mask='on')

In [ ]:
mode = 'stack'
plot_array(src, mode)

In [ ]:
if array_index == 1:
    chan = np.split(array, array_index, axis=2)
if array_index == 2:
    c1, c2 = np.split(array, array_index, axis=2)
if array_index == 3:
    c1, c2, c3 = np.split(array, array_index, axis=2)
if array_index == 4:
    c1, c2, c3, c4 = np.split(array, array_index, axis=2)
if array_index == 5:
    c1, c2, c3, c4, c5 = np.split(array, array_index, axis=2)
if array_index == 6:
    c1, c2, c3, c4, c6 = np.split(array, array_index, axis=2)
if array_index == 7:
    c1, c2, c3, c4, c6, c7 = np.split(array, array_index, axis=2)
if array_index == 8:
    c1, c2, c3, c4, c6, c7, c8 = np.split(array, array_index, axis=2)
if array_index == 9:
    c1, c2, c3, c4, c6, c7, c8, c9 = np.split(array, array_index, axis=2)
if array_index == 10:
    c1, c2, c3, c4, c6, c7, c8, c9, c10 = np.split(array, array_index, axis=2)
if array_index == 11:
    c1, c2, c3, c4, c6, c7, c8, c9, c10, c11 = np.split(array, array_index, axis=2)
if array_index == 12:
    c1, c2, c3, c4, c6, c7, c8, c9, c10, c11, c12 = np.split(array, array_index, axis=2)
if array_index == 13:
    c1, c2, c3, c4, c6, c7, c8, c9, c10, c11, c12, c13 = np.split(array, array_index, axis=2)
if array_index == 14:
    c1, c2, c3, c4, c6, c7, c8, c9, c10, c11, c12, c13, c14 = np.split(array, array_index, axis=2)

In [ ]:
file = r'C:\Users\einar\Desktop\omi\stack\plate3_B05_016.npy'
array = np.load(file)
array_index = array.shape[2]
print(array.shape[2])
parent_mask_arr_ls = np.unique(parent_mask_arr)
p_ls = len(parent_mask_arr_ls)
array_dim_ls = list(range(1, p_ls))

print(array_dim_ls)

for i in range(0, array_index):
    print(i)
    chan = 'c'+i
    chan = np.split(array, i, axis=2)

In [ ]:
# Function to pad a single cell array with 0s to a specified height x width. 
# This function is used within the next function.
def padding(array, height, width):
    h = array.shape[0]
    w = array.shape[1]
    
    aa = (height - h) // 2
    aaa = height - aa - h

    bb = (width - w) // 2
    bbb = width - bb - w
    return np.pad(array, pad_width=((aa, aaa), (bb, bbb)), mode='constant')
                                            
#Function that takes two cellpose mask tifs as input and outputs 
#cropped and padded images of each uneque object in the mask.
#Useful if using cell mask and nucleus mask.
def crop_mask_from_mask_parent_child_img(src, parent_obj, child_obj, chan, height, width, parent_size_cutoff, child_size_cutoff, parent_vs_child_cutoff, nr_of_dims, nr_of_channels_to_crop, nuclei_dim, cell_dim, obj3_dim):                        
    if os.path.exists(src+'/cropped_mask_'+parent_obj) is False:         
        os.mkdir(src+'/cropped_mask_'+parent_obj)
    if nuclei_dim == cell_dim == obj3_dim:
        print('nuclei dim, cell_dim and obj3_dim cannot be the same dimension')
    if nuclei_dim == cell_dim:
        print('nuclei dim and cell_dim cannot be the same dimension')
    if cell_dim == obj3_dim:
        print('cell dim and obj3_dim cannot be the same dimension')
    if nuclei_dim == obj3_dim:
        print('nuclei dim and obj3_dim cannot be the same dimension')
    if os.path.exists(src+'/cropped_mask_'+parent_obj) is False:         
        os.mkdir(src+'/cropped_mask_'+parent_obj)
    for root, _, files in os.walk(src+'/'+mode):
        for file in files:
            name = os.path.splitext(file)[0]
            extension = os.path.splitext(file)[1]
            test_name = os.path.join(src+'/cropped_mask_'+parent_obj, name+'_'+'1'+'_'+parent_obj+'.npy')
            #test_mask_name = os.path.join(src+'/'+obj+'/',name+'.tif')
            if extension != '.npy':
                print(file+': is not npy')
            else:
                array = np.load(file)
                array_index = array.shape[2]
                print(array_index)
                if array_index == nr_of_dims == False:
                    print(file+': no masks')
                    pass
                else:
                    if os.path.exists(test_name) == True:
                        print(file+': '+obj+' already cropped from mask')
                        pass
                    else:
                        if array_index == 1:
                            c1 = np.split(array, array_index, axis=2)
                            c1 = np.squeeze(c1, 2)
                        if array_index == 2:
                            c1, c2 = np.split(array, array_index, axis=2)
                            c1 = np.squeeze(c1, 2)
                            c1 = np.squeeze(c1, 2)
                        if array_index == 3:
                            c1, c2, c3 = np.split(array, array_index, axis=2)
                            c1 = np.squeeze(c1, 2)
                            c1 = np.squeeze(c1, 2)
                            c1 = np.squeeze(c1, 2)
                        if array_index == 4:
                            c1, c2, c3, c4 = np.split(array, array_index, axis=2)
                            c1 = np.squeeze(c1, 2)
                            c1 = np.squeeze(c1, 2)
                            c1 = np.squeeze(c1, 2)
                            c1 = np.squeeze(c1, 2)
                            
                        if array_index == 5:
                            c1, c2, c3, c4, c5 = np.split(array, array_index, axis=2)
                        if array_index == 6:
                            c1, c2, c3, c4, c5, c6 = np.split(array, array_index, axis=2)
                        if array_index == 7:
                            c1, c2, c3, c4, c5, c6, c7 = np.split(array, array_index, axis=2)
                            c1 = np.squeeze(c1, 2)
                            c2 = np.squeeze(c2, 2)
                            c3 = np.squeeze(c3, 2)
                            c4 = np.squeeze(c4, 2)
                            c5 = np.squeeze(c5, 2)
                            c6 = np.squeeze(c6, 2)
                            c7 = np.squeeze(c7, 2)
                            
                        if array_index == 8:
                            c1, c2, c3, c4, c5, c6, c7, c8 = np.split(array, array_index, axis=2)
                        if array_index == 9:
                            c1, c2, c3, c4, c5, c6, c7, c8, c9 = np.split(array, array_index, axis=2)
                        if array_index == 10:
                            c1, c2, c3, c4, c5, c6, c7, c8, c9, c10 = np.split(array, array_index, axis=2)
                        if array_index == 11:
                            c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11 = np.split(array, array_index, axis=2)
                        if array_index == 12:
                            c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12 = np.split(array, array_index, axis=2)
                        if array_index == 13:
                            c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13 = np.split(array, array_index, axis=2)
                        if array_index == 14:
                            c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14 = np.split(array, array_index, axis=2)
                        if nr_of_channels_to_crop == 3:
                            img_arr = np.stack([c1, c2, c3], axis=2)
                            img_arr = np.squeeze(img_arr)
                        if nr_of_channels_to_crop == 4:
                            img_arr = np.stack([c1, c2, c3, c4], axis=2)
                            img_arr = np.squeeze(img_arr)
                        if cell_dim == 1:
                            mask_arr = c1
                        if cell_dim == 2:
                            mask_arr = c2
                        if cell_dim == 3:
                            mask_arr = c3
                        if cell_dim == 4:
                            mask_arr = c5
                        if cell_dim == 5:
                            mask_arr = c5
                        if cell_dim == 6:
                            mask_arr = c6
                        if cell_dim == 7:
                            mask_arr = c7
                        if cell_dim == 8:
                            mask_arr = c8
                        if cell_dim == 9:
                            mask_arr = c9
                        if cell_dim == 10:
                            mask_arr = c10
                        if cell_dim == 11:
                            mask_arr = c11
                        if cell_dim == 12:
                            mask_arr = c12
                        if cell_dim == 13:
                            mask_arr = c13
                        if cell_dim == 14:
                            mask_arr = c14
                        if nuclei_dim != 'off':
                            if nuclei_dim == 1:
                                mask2_arr = c1
                            if nuclei_dim == 2:
                                mask2_arr = c2
                            if nuclei_dim == 3:
                                mask2_arr = c3
                            if nuclei_dim == 4:
                                mask2_arr = c4
                            if nuclei_dim == 5:
                                mask2_arr = c5
                            if nuclei_dim == 6:
                                mask2_arr = c6
                            if nuclei_dim == 7:
                                mask2_arr = c7
                            if nuclei_dim == 8:
                                mask2_arr = c8
                            if nuclei_dim == 9:
                                mask2_arr = c9
                            if nuclei_dim == 10:
                                mask2_arr = c10
                            if nuclei_dim == 11:
                                mask2_arr = c11
                            if nuclei_dim == 12:
                                mask2_arr = c12
                            if nuclei_dim == 13:
                                mask2_arr = c13
                            if nuclei_dim == 14:
                                mask2_arr = c14
                        if obj3_dim != 'off':
                            if obj3_dim == 1:
                                mask3_arr = c1
                            if obj3_dim == 2:
                                mask3_arr = c2
                            if obj3_dim == 3:
                                mask3_arr = c3
                            if obj3_dim == 4:
                                mask3_arr = c4
                            if obj3_dim == 5:
                                mask3_arr = c5
                            if obj3_dim == 6:
                                mask3_arr = c6
                            if obj3_dim == 7:
                                mask3_arr = c7
                            if obj3_dim == 8:
                                mask3_arr = c8
                            if obj3_dim == 9:
                                mask3_arr = c9
                            if obj3_dim == 10:
                                mask3_arr = c10
                            if obj3_dim == 11:
                                mask3_arr = c11
                            if obj3_dim == 12:
                                mask3_arr = c12
                            if obj3_dim == 13:
                                mask3_arr = c13
                            if obj3_dim == 14:
                                mask3_arr = c14

                            parent_mask_arr = mask_arr
                            child_mask_arr = mask2_arr
                            child_mask_arr_2 = child_mask_arr + 1
                            
                            child2_mask_arr = mask3_arr
                            child2_mask_arr_2 = child2_mask_arr + 1

                            parent_mask_arr_ls = np.unique(parent_mask_arr)
                            p_ls = len(parent_mask_arr_ls)
                            parent_mask_arr_ls = list(range(1, p_ls))
                            
                            print(parent_mask_arr_ls)

                            for i in parent_mask_arr_ls:
                                print(i)
                                parent_mask_id_nr = i
                                parent_obj_nr = str(i)
                                parent_single_obj_mask = np.where(parent_mask_arr == parent_mask_id_nr, 1, 0)
                                parent_gray = np.multiply(parent_mask_arr, parent_single_obj_mask)

                                child_gray = np.multiply(child_mask_arr, parent_single_obj_mask)
                                child_gray_2 = np.multiply(child_mask_arr_2, parent_single_obj_mask)

                                parent_gray = np.delete(parent_gray,np.where(~parent_gray.any(axis=0))[0], axis=1)
                                parent_gray = np.transpose(parent_gray)
                                parent_gray = np.delete(parent_gray,np.where(~parent_gray.any(axis=0))[0], axis=1)
                                parent_gray = np.transpose(parent_gray)
                                parent_gray_h, parent_gray_w = parent_gray.shape

                                child_gray = np.delete(child_gray,np.where(~child_gray.any(axis=0))[0], axis=1)
                                child_gray = np.transpose(child_gray)
                                child_gray = np.delete(child_gray,np.where(~child_gray.any(axis=0))[0], axis=1)
                                child_gray = np.transpose(child_gray)

                                child_gray_2 = np.delete(child_gray_2,np.where(~child_gray_2.any(axis=0))[0], axis=1)
                                child_gray_2 = np.transpose(child_gray_2)
                                child_gray_2 = np.delete(child_gray_2,np.where(~child_gray_2.any(axis=0))[0], axis=1)
                                child_gray_2 = np.transpose(child_gray_2)

                                print(parent_obj, parent_gray.shape)
                                print(child_obj, child_gray_2.shape)
                                print(child_obj, child_gray.shape)
                                
                                r, g, b, a = np.split(img_arr, 4, axis=2) # cahnged from cv2.imread
                                #maxes out memory here
                                b = np.multiply(b, parent_single_obj_mask)
                                g = np.multiply(g, parent_single_obj_mask)
                                r = np.multiply(r, parent_single_obj_mask)
                                a = np.multiply(a, parent_single_obj_mask)

                                b = np.delete(b,np.where(~b.any(axis=0))[0], axis=1)
                                g = np.delete(g,np.where(~g.any(axis=0))[0], axis=1)
                                r = np.delete(r,np.where(~r.any(axis=0))[0], axis=1)
                                a = np.delete(r,np.where(~a.any(axis=0))[0], axis=1)

                                b = np.transpose(b)
                                g = np.transpose(g)
                                r = np.transpose(r)
                                a = np.transpose(a)

                                b = np.delete(b,np.where(~b.any(axis=0))[0], axis=1)
                                g = np.delete(g,np.where(~g.any(axis=0))[0], axis=1)
                                r = np.delete(r,np.where(~r.any(axis=0))[0], axis=1)
                                a = np.delete(r,np.where(~a.any(axis=0))[0], axis=1)

                                b = np.transpose(b)
                                g = np.transpose(g)
                                r = np.transpose(r)
                                a = np.transpose(a)
                                
                                b_h, b_w = b.shape
                                print(b.shape)

                                if parent_gray_h > height:
                                    print(parent_obj, ' img heigt is >>> '+str(height))
                                    dif = parent_gray_h - height
                                    rem_start = dif/2
                                    rem_end = parent_gray_h - rem_start
                                    rem_start = int(rem_start)
                                    rem_end = int(rem_end)
                                    print('img dim cut: start @ row '+str(rem_start)+', end @ row: '+str(rem_end))

                                    child_gray_2 = np.delete(child_gray_2, slice(rem_start, rem_end), axis=0)
                                    child_gray = np.delete(child_gray, slice(rem_start, rem_end), axis=0)
                                    parent_gray = np.delete(parent_gray, slice(rem_start, rem_end), axis=0)
                                    
                                    b = np.delete(b, slice(rem_start, rem_end), axis=0)
                                    g = np.delete(g, slice(rem_start, rem_end), axis=0)
                                    r = np.delete(r, slice(rem_start, rem_end), axis=0)
                                    a = np.delete(a, slice(rem_start, rem_end), axis=0)
                                    
                                else:
                                    print(parent_obj,' img heigt is <<< '+str(height))
                                    parent_gray_w = parent_gray.shape[1]
                                    if parent_gray_w > width:
                                        print('img width is >>> '+str(width))
                                        parent_gray = np.transpose(parent_gray)
                                        parent_gray_w = parent_gray.shape[1]
                                        dif = parent_gray_w - width
                                        rem_start = dif/2
                                        rem_end = parent_gray_w - rem_start
                                        rem_start = int(rem_start)
                                        rem_end = int(rem_end)
                                        print(parent_obj,' img dim cut: start @ col '+str(rem_start)+', end @ col: '+str(rem_end))

                                        parent_gray = np.delete(parent_gray, slice(rem_start, rem_end), axis=0)
                                        parent_gray = np.transpose(parent_gray)

                                        child_gray_2 = np.transpose(child_gray_2)
                                        child_gray_2 = np.delete(child_gray_2, slice(rem_start, rem_end), axis=0)
                                        child_gray_2 = np.transpose(child_gray_2)

                                        child_gray = np.transpose(child_gray)
                                        child_gray = np.delete(child_gray, slice(rem_start, rem_end), axis=0)
                                        child_gray = np.transpose(child_gray)
                                        
                                        b = np.delete(b, slice(rem_start, rem_end), axis=0)
                                        g = np.delete(g, slice(rem_start, rem_end), axis=0)
                                        r = np.delete(r, slice(rem_start, rem_end), axis=0)
                                        a = np.delete(a, slice(rem_start, rem_end), axis=0)

                                        b = np.transpose(b)
                                        g = np.transpose(g)
                                        r = np.transpose(r)
                                        a = np.transpose(a)

                                    else:
                                        print(parent_obj,' img width is <<< '+str(width))

                                        child_gray_2_h, child_gray_2_w = child_gray_2.shape
                                        child_gray_2_ima = child_gray_2_h*child_gray_2_w
                                        if child_gray_2_ima <= child_size_cutoff:
                                            print(child_obj,': is smaler than',child_size_cutoff)
                                            pass
                                        else:
                                            parent_gray_img_h, parent_gray_img_w = parent_gray.shape
                                            parent_gray_ima = parent_gray_img_h*parent_gray_img_w
                                            if parent_gray_ima <= parent_size_cutoff:
                                                print(parent_obj,': is smaler than', parent_size_cutoff)
                                                pass
                                            else:
                                                parent_vs_child = child_gray_2_ima/parent_gray_ima
                                                if parent_vs_child <= parent_vs_child_cutoff:
                                                    print(child_obj,': is more than half of ',parent_obj)
                                                    pass
                                                else:
                                                
                                                    b = padding(b, height, width)
                                                    g = padding(g, height, width)
                                                    r = padding(r, height, width)
                                                    a = padding(a, height, width)

                                                    if  b.shape ==  g.shape ==  r.shape == a.shape:
                                                        rgba_pad = cv2.merge((b,g,r,a)).astype(bitdepth)

                                                        parent_gray_pad = padding(parent_gray, height, width)
                                                        print(parent_gray.shape)
                                                        child_gray_pad = padding(child_gray, height, width)
                                                        print(child_gray.shape)

                                                        parent_obj_path = os.path.join(src_mask+'/'+parent_obj+'/cropped_mask_'+parent_obj, name+'_'+parent_obj_nr+'_'+parent_obj+'.tif')
                                                        child_obj_path = os.path.join(src_mask+'/'+child_obj+'/cropped_mask_'+child_obj, name+'_'+parent_obj_nr+'_'+child_obj+'.tif')
                                                        object_path = os.path.join(src_mask+'/'+parent_obj+'/cropped_img_'+parent_obj+'_'+chan, name+'_'+parent_obj_nr+'_'+chan+'.png')

                                                        cv2.imwrite(parent_obj_path, parent_gray_pad)
                                                        print('Cropped: '+parent_obj_path)
                                                        cv2.imwrite(child_obj_path, child_gray_pad)
                                                        print('Cropped: '+child_obj_path)
                                                        cv2.imwrite(object_path, rgba_pad)
                                                        print('Cropped: '+object_path)
                                                    else:
                                                        print('rgb chanel shape mismatch')


In [ ]:
src = r'C:\Users\einar\Desktop\omi'

parent_obj = 'cell'
child_obj = 'nuclei'
chan = 'stack'
height = 448
width = 448
parent_size_cutoff = 2000
child_size_cutoff = 1000
parent_vs_child_cutoff = 0.5
nr_of_dims = 7
nr_of_channels_to_crop = 4
nuclei_dim = 5
cell_dim = 6
obj3_dim = 7
crop_mask_from_mask_parent_child_img(src, parent_obj, child_obj, chan, height, width, parent_size_cutoff, child_size_cutoff, parent_vs_child_cutoff, nr_of_dims, nr_of_channels_to_crop, nuclei_dim, cell_dim, obj3_dim)

In [ ]:
# Function to pad a single cell array with 0s to a specified height x width. 
# This function is used within the next function.
def padding(array, height, width):
    h = array.shape[0]
    w = array.shape[1]
    
    aa = (height - h) // 2
    aaa = height - aa - h

    bb = (width - w) // 2
    bbb = width - bb - w
    return np.pad(array, pad_width=((aa, aaa), (bb, bbb)), mode='constant')


#Function that takes two cellpose mask tifs as input and outputs 
#cropped and padded images of each uneque object in the mask.
#Useful if using cell mask and nucleus mask.
def crop_mask_from_mask_parent_child_img(src, src_mask, parent_obj, child_obj, chan, height, width, parent_size_cutoff, child_size_cutoff, parent_vs_child_cutoff):                        
    if os.path.exists(src_mask+'/'+parent_obj+'/cropped_img_'+parent_obj+'_'+chan) is False:         
        os.mkdir(src_mask+'/'+parent_obj+'/cropped_img_'+parent_obj+'_'+chan)
    if os.path.exists(src_mask+'/'+parent_obj+'/cropped_mask_'+parent_obj) is False:         
        os.mkdir(src_mask+'/'+parent_obj+'/cropped_mask_'+parent_obj)
    if os.path.exists(src_mask+'/'+child_obj+'/cropped_mask_'+child_obj) is False:         
        os.mkdir(src_mask+'/'+child_obj+'/cropped_mask_'+child_obj)
    for root, _, files in os.walk(src+'/'+chan):
        for file in files:
            name = os.path.splitext(file)[0]
            extension = os.path.splitext(file)[1]
            test_name = os.path.join(src_mask+'/'+parent_obj+'/cropped_mask_'+parent_obj, name+'_'+'1'+'_'+parent_obj+'.tif')
            test_parent_mask_name = os.path.join(src_mask+'/'+parent_obj+'/',name+'.tif')
            test_child_mask_name = os.path.join(src_mask+'/'+child_obj+'/',name+'.tif')
            test_img_name = os.path.join(src+'/'+chan+'/',name+'.png')
            if extension == '.tif':
                print(file+': Run rescale intensity first')
                pass
            elif extension == '.png':
                print(name+': is a png')
                if os.path.exists(test_parent_mask_name) == False:
                    print(file+': has no '+parent_obj+' mask')
                    pass
                else:
                    if os.path.exists(test_child_mask_name) == False:
                        print(file+': has no '+child_obj+' mask')
                        pass
                    else:
                        if os.path.exists(test_name) == True:
                            print(file+': '+parent_obj+' already cropped from mask')
                            pass
                        else:
                            parent = os.path.join(src_mask+'/'+parent_obj, name+'.tif')
                            child = os.path.join(src_mask+'/'+child_obj, name+'.tif')
                            img = os.path.join(src+'/'+chan, name+'.png')

                            img_arr = cv2.imread(img, -1)
                            parent_mask_arr = cv2.imread(parent, -1)
                            child_mask_arr = cv2.imread(child, -1)
                            child_mask_arr_2 = child_mask_arr + 1

                            parent_mask_arr_ls = np.unique(parent_mask_arr)
                            print(parent_mask_arr_ls)

                            for i in parent_mask_arr_ls:
                                parent_mask_id_nr = i
                                parent_obj_nr = str(i)
                                parent_mask_id_nr = i
                                parent_single_obj_mask = np.where(parent_mask_arr == parent_mask_id_nr, 1, 0)
                                parent_gray = np.multiply(parent_mask_arr, parent_single_obj_mask)

                                child_gray = np.multiply(child_mask_arr, parent_single_obj_mask)
                                child_gray_2 = np.multiply(child_mask_arr_2, parent_single_obj_mask)

                                parent_gray = np.delete(parent_gray,np.where(~parent_gray.any(axis=0))[0], axis=1)
                                parent_gray = np.transpose(parent_gray)
                                parent_gray = np.delete(parent_gray,np.where(~parent_gray.any(axis=0))[0], axis=1)
                                parent_gray = np.transpose(parent_gray)
                                parent_gray_h, parent_gray_w = parent_gray.shape

                                child_gray = np.delete(child_gray,np.where(~child_gray.any(axis=0))[0], axis=1)
                                child_gray = np.transpose(child_gray)
                                child_gray = np.delete(child_gray,np.where(~child_gray.any(axis=0))[0], axis=1)
                                child_gray = np.transpose(child_gray)

                                child_gray_2 = np.delete(child_gray_2,np.where(~child_gray_2.any(axis=0))[0], axis=1)
                                child_gray_2 = np.transpose(child_gray_2)
                                child_gray_2 = np.delete(child_gray_2,np.where(~child_gray_2.any(axis=0))[0], axis=1)
                                child_gray_2 = np.transpose(child_gray_2)

                                print(parent_obj, parent_gray.shape)
                                print(child_obj, child_gray_2.shape)
                                print(child_obj, child_gray.shape)
                                
                                b, g, r = cv2.split(img_arr.astype(bitdepth))
                                
                                b = np.multiply(b, parent_single_obj_mask)
                                g = np.multiply(g, parent_single_obj_mask)
                                r = np.multiply(r, parent_single_obj_mask)

                                b = np.delete(b,np.where(~b.any(axis=0))[0], axis=1)
                                g = np.delete(g,np.where(~g.any(axis=0))[0], axis=1)
                                r = np.delete(r,np.where(~r.any(axis=0))[0], axis=1)

                                b = np.transpose(b)
                                g = np.transpose(g)
                                r = np.transpose(r)

                                b = np.delete(b,np.where(~b.any(axis=0))[0], axis=1)
                                g = np.delete(g,np.where(~g.any(axis=0))[0], axis=1)
                                r = np.delete(r,np.where(~r.any(axis=0))[0], axis=1)

                                b = np.transpose(b)
                                g = np.transpose(g)
                                r = np.transpose(r)
                                
                                b_h, b_w = b.shape
                                print(b.shape)

                                if parent_gray_h > height:
                                    print(parent_obj, ' img heigt is >>> '+str(height))
                                    dif = parent_gray_h - height
                                    rem_start = dif/2
                                    rem_end = parent_gray_h - rem_start
                                    rem_start = int(rem_start)
                                    rem_end = int(rem_end)
                                    print('img dim cut: start @ row '+str(rem_start)+', end @ row: '+str(rem_end))

                                    child_gray_2 = np.delete(child_gray_2, slice(rem_start, rem_end), axis=0)
                                    child_gray = np.delete(child_gray, slice(rem_start, rem_end), axis=0)
                                    parent_gray = np.delete(parent_gray, slice(rem_start, rem_end), axis=0)
                                    
                                    b = np.delete(b, slice(rem_start, rem_end), axis=0)
                                    g = np.delete(g, slice(rem_start, rem_end), axis=0)
                                    r = np.delete(r, slice(rem_start, rem_end), axis=0)
                                    
                                else:
                                    print(parent_obj,' img heigt is <<< '+str(height))
                                    parent_gray_w = parent_gray.shape[1]
                                    if parent_gray_w > width:
                                        print('img width is >>> '+str(width))
                                        parent_gray = np.transpose(parent_gray)
                                        parent_gray_w = parent_gray.shape[1]
                                        dif = parent_gray_w - width
                                        rem_start = dif/2
                                        rem_end = parent_gray_w - rem_start
                                        rem_start = int(rem_start)
                                        rem_end = int(rem_end)
                                        print(parent_obj,' img dim cut: start @ col '+str(rem_start)+', end @ col: '+str(rem_end))

                                        parent_gray = np.delete(parent_gray, slice(rem_start, rem_end), axis=0)
                                        parent_gray = np.transpose(parent_gray)

                                        child_gray_2 = np.transpose(child_gray_2)
                                        child_gray_2 = np.delete(child_gray_2, slice(rem_start, rem_end), axis=0)
                                        child_gray_2 = np.transpose(child_gray_2)

                                        child_gray = np.transpose(child_gray)
                                        child_gray = np.delete(child_gray, slice(rem_start, rem_end), axis=0)
                                        child_gray = np.transpose(child_gray)
                                        
                                        b = np.delete(b, slice(rem_start, rem_end), axis=0)
                                        g = np.delete(g, slice(rem_start, rem_end), axis=0)
                                        r = np.delete(r, slice(rem_start, rem_end), axis=0)

                                        b = np.transpose(b)
                                        g = np.transpose(g)
                                        r = np.transpose(r)

                                    else:
                                        print(parent_obj,' img width is <<< '+str(width))

                                        child_gray_2_h, child_gray_2_w = child_gray_2.shape
                                        child_gray_2_ima = child_gray_2_h*child_gray_2_w
                                        if child_gray_2_ima <= child_size_cutoff:
                                            print(child_obj,': is smaler than',child_size_cutoff)
                                            pass
                                        else:
                                            parent_gray_img_h, parent_gray_img_w = parent_gray.shape
                                            parent_gray_ima = parent_gray_img_h*parent_gray_img_w
                                            if parent_gray_ima <= parent_size_cutoff:
                                                print(parent_obj,': is smaler than', parent_size_cutoff)
                                                pass
                                            else:
                                                parent_vs_child = child_gray_2_ima/parent_gray_ima
                                                if parent_vs_child <= parent_vs_child_cutoff:
                                                    print(child_obj,': is more than half of ',parent_obj)
                                                    pass
                                                else:
                                                
                                                    b = padding(b, height, width)
                                                    g = padding(g, height, width)
                                                    r = padding(r, height, width)

                                                    if  b.shape ==  g.shape ==  r.shape:
                                                        rgb_pad = cv2.merge((b,g,r)).astype(bitdepth)

                                                        parent_gray_pad = padding(parent_gray, height, width)
                                                        print(parent_gray.shape)
                                                        child_gray_pad = padding(child_gray, height, width)
                                                        print(child_gray.shape)

                                                        parent_obj_path = os.path.join(src_mask+'/'+parent_obj+'/cropped_mask_'+parent_obj, name+'_'+parent_obj_nr+'_'+parent_obj+'.tif')
                                                        child_obj_path = os.path.join(src_mask+'/'+child_obj+'/cropped_mask_'+child_obj, name+'_'+parent_obj_nr+'_'+child_obj+'.tif')
                                                        object_path = os.path.join(src_mask+'/'+parent_obj+'/cropped_img_'+parent_obj+'_'+chan, name+'_'+parent_obj_nr+'_'+chan+'.png')

                                                        cv2.imwrite(parent_obj_path, parent_gray_pad)
                                                        print('Cropped: '+parent_obj_path)
                                                        cv2.imwrite(child_obj_path, child_gray_pad)
                                                        print('Cropped: '+child_obj_path)
                                                        cv2.imwrite(object_path, rgb_pad)
                                                        print('Cropped: '+object_path)
                                                    else:
                                                        print('rgb chanel shape mismatch')

def objects_to_plate_well_field_folder(src_mask, obj, chan):
    if os.path.exists(src_mask+'/'+obj+'/montage') is False:
        os.mkdir(src_mask+'/'+obj+'/montage')
    if os.path.exists(src_mask+'/'+obj+'/montage/'+chan) is False:
        os.mkdir(src_mask+'/'+obj+'/montage/'+chan)
    for root, _, files in os.walk(src_mask+'/'+obj+'/cropped_'+obj+'_'+chan):
        for file in files:
            ext = os.path.splitext(file)[1]
            img_name = os.path.splitext(file)[0]
            img_path = os.path.join(src_mask+'/'+obj+'/cropped_'+obj+'_'+chan, file)
            if ext == '.png':
                    split = img_name.split('_')
                    plateid = split[0]
                    wellid = split[1]
                    fieldid = int(split[2])
                    fieldid_str = str(split[2])
                    objectid = int(split[3])
                    plate_well_field = plateid+'_'+wellid+'_'+fieldid_str
                    
                    if os.path.exists(src_mask+'/'+obj+'/montage/'+chan+'/'+plate_well_field) is False:
                        os.mkdir(src_mask+'/'+obj+'/montage/'+chan+'/'+plate_well_field)
                    new_path = os.path.join(src_mask+'/'+obj+'/montage/'+chan+'/'+plate_well_field, file)
                    shutil.move(img_path, new_path)                   
                    print(file,': mooved')
                    
                    if os.path.exists(src_mask+'/'+obj+'/montage/'+chan+'/'+plate_well_field+'/img_metadata.csv') is False:
                        with open(src_mask+'/'+obj+'/montage/'+chan+'/'+plate_well_field+'/img_metadata.csv', 'w') as csvfile:
                            img_metadata = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                            img_metadata.writerow(['img_path', 'new_path', 'plateid', 'wellid', 'fieldid', 'objectid'])
                    img_metadata_csv = src_mask+'/'+obj+'/montage/'+chan+'/'+plate_well_field+'/img_metadata.csv'
                    new_row = [img_path, new_path, plateid, wellid, fieldid, objectid]
                    with open(img_metadata_csv, 'a+', newline='') as write_obj:
                        csv_writer = csv.writer(write_obj)
                        csv_writer.writerow(new_row)
                        
def make_montag(src_mask, obj, chan):
    for folders, subfolders, files in os.walk(src_mask+'/'+obj+'/montage/'+chan):
        for subfolder in subfolders:
            if os.path.exists(src_mask+'/'+obj+'/montage/'+chan+'/'+subfolder+'/img_metadata.csv') is True:
                img_metadata = src_mask+'/'+obj+'/montage/'+chan+'/'+subfolder+'/img_metadata.csv'
                df = pd.read_csv(img_metadata)
                dim = (int(len(df['new_path'])/2)+1)
                df_sorted = df.sort_values(by=['fieldid', 'objectid'])
                img1_path = os.path.abspath(df_sorted['new_path'].iloc[0])
                print('img1:',img1_path)
                img1 = cv2.imread(img1_path, -1)
                montag_path = src_mask+'/'+obj+'/montage/'+chan+'/'+subfolder+'.png'
                for num, name in enumerate(df_sorted['img_path'], start=1):
                    if name.endswith('.png') == True:
                        try:
                            if os.path.exists(src_mask+'/'+obj+'/montage/'+chan+'/'+subfolder+'.png') is False:
                                img_concat_path = os.path.abspath(df_sorted['new_path'].iloc[num])
                                print(img_concat_path, num)
                                img_concat = cv2.imread(img_concat_path, -1)
                                montage = cv2.hconcat([img1, img_concat])
                                cv2.imwrite(montag_path, montage)
                            else:
                                img_concat_path = os.path.abspath(df_sorted['new_path'].iloc[num])
                                print(img_concat_path, num)
                                img_concat = cv2.imread(img_concat_path, -1)
                                montage = cv2.imread(montag_path, -1)
                                montage = cv2.hconcat([montage, img_concat])
                                cv2.imwrite(montag_path, montage)
                        except:
                            print(name,': failed')
                    else:
                        print('missing metadata file')

def measure_object_intensities(src_mask):
    if os.path.exists(src_mask+'/'+obj+'/cropped_'+obj+'/cell_intensity.csv') is False:         
        with open(src_mask+'/'+obj+'/cropped_'+obj+'/cell_intensity.csv', 'w') as csvfile:
            cell_intensity = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            cell_intensity.writerow(['file_name','chan_2_Q95','chan_3_Q95','chan_4_Q95'])
    for root, _, files in os.walk(src_mask+'/'+obj+'/cropped_'+obj):
        print('file_name','chan_2_Q95','chan_3_Q95','chan_4_Q95')
        for file in files:
            
            extension=os.path.splitext(file)[1]
            if extension == '.png' == False:
                print(file+': is not png')
            else:
                img = os.path.join(src_mask+'/'+obj+'/cropped_'+obj, file)
                img_arr = cv2.imread(img, -1)
                b, g, r = cv2.split(img_arr.astype(bitdepth))

                chan_4_Q95 = np.quantile(b, (.95))
                chan_3_Q95 = np.quantile(g, (.95))
                chan_2_Q95 = np.quantile(r, (.95))

                #chan_4_median = np.median(b)
                #chan_3_median = np.median(g) 
                #chan_2_median = np.median(r)

                #chan_4_mean = np.mean(b)
                #chan_3_mean = np.mean(g)
                #chan_2_mean = np.mean(r)

                new_row = [file, chan_2_Q95, chan_3_Q95, chan_4_Q95]
                cell_intensity_csv = src_mask+'/'+obj+'/cropped_'+obj+'/cell_intensity.csv'
                print(file, chan_2_Q95, chan_3_Q95, chan_4_Q95)
                with open(cell_intensity_csv, 'a+', newline='') as write_obj:
                    csv_writer = csv.writer(write_obj)
                    csv_writer.writerow(new_row)
                
def classify_cells_intensity(src_mask):
    if os.path.exists(src_mask+'/'+obj+'/cropped_'+obj+'/no_nucleus') is False:         
        os.mkdir(src_mask+'/'+obj+'/cropped_'+obj+'/no_nucleus')
    if os.path.exists(src_mask+'/'+obj+'/cropped_'+obj+'/no_signal') is False:         
        os.mkdir(src_mask+'/'+obj+'/cropped_'+obj+'/no_signal')
    if os.path.exists(src_mask+'/'+obj+'/cropped_'+obj+'/infected') is False:         
        os.mkdir(src_mask+'/'+'cells/cropped_cells/infected')
    if os.path.exists(src_mask+'/'+obj+'/cropped_'+obj+'/not_infected') is False:         
        os.mkdir(src_mask+'/'+obj+'/cropped_'+obj+'/not_infected')
    #if os.path.exists(src_mask+'/'+'cells/cropped_cells/artifact') is False:         
    #    os.mkdir(src_mask+'/'+'cells/cropped_cells/artifact')
    if os.path.exists(src_mask+'/'+obj+'/cropped_'+obj+'/cell_intensity.csv') is False:
        print ('cell_intensity.csv does not exist')
    else:
        print ('cell_intensity.csv exist')
        for root, _, files in os.walk(src_mask+'/'+obj+'/cropped_'+obj):
            
            cell_intensity_csv = src_mask+'/'+obj+'/cropped_'+obj+'/cell_intensity.csv'                
            df = pd.read_csv(cell_intensity_csv)
            chan_4_Q95 = df['chan_4_Q95'].quantile(0.5)
            chan_4_std = df['chan_4_Q95'].std()
            chan_3_Q95 = df['chan_3_Q95'].quantile(0.5)
            chan_3_std = df['chan_3_Q95'].std()                
            chan_2_Q95 = df['chan_2_Q95'].quantile(0.5)
            chan_2_std = df['chan_2_Q95'].std()
                
            chan_4_mean_std = 500
            chan_3_mean_std = 2000
            chan_2_mean_std = 2000
            
            for file in files:
                extension=os.path.splitext(file)[1]
                if extension == '.png' == False:
                    print(file+': is not png')
                else:
                    no_nucleus_path = os.path.join(src_mask+'/'+obj+'/cropped_'+obj+'/no_nucleus', file)
                    no_signal_path = os.path.join(src_mask+'/'+obj+'/cropped_'+obj+'/no_nucleus', file)
                    noninfected_path = os.path.join(src_mask+'/'+obj+'/cropped_'+obj+'/not_infected', file)
                    infected_path = os.path.join(src_mask+'/'+obj+'/cropped_'+obj+'/infected', file)
                    #artifact = os.path.join(src_mask+'/'+'cells/cropped_cells/infected', file)
                    img = os.path.join(src_mask+'/'+obj+'/cropped_'+obj, file)
                    print(img)
                    
                    img_arr = cv2.imread(img, -1)
                    b, g, r = cv2.split(img_arr)
                    
                    chan_4_cell_Q95 = np.quantile(b, (0.95))
                    chan_3_cell_Q95 = np.quantile(g, (0.99))
                    chan_2_cell_Q95 = np.quantile(r, (0.95))
                    
                    print(file, chan_4_cell_Q95, chan_3_cell_Q95, chan_2_cell_Q95, chan_4_mean_std, chan_3_mean_std, chan_2_mean_std)

                    if chan_4_cell_Q95 <= chan_4_mean_std:
                        shutil.move(img, no_nucleus_path)
                    else:
                        if chan_2_cell_Q95 <= chan_2_mean_std:
                            shutil.move(img, no_signal_path)
                        else:
                            if chan_3_cell_Q95 < chan_3_mean_std:
                                shutil.move(img, noninfected_path)
                            else:
                                if chan_3_cell_Q95 >= chan_3_mean_std:
                                    shutil.move(img, infected_path)

#This function archives the raw data and other intermediate files.
def tardir(src):
    name = os.path.basename(src)
    tar_name_1 = name+'_tar.gz'
    tar_name_2 = name+'rgb_png_tar.gz'
    tar_name_3 = name+'frgb_png_tar.gz'
    os.chdir(src)
    with tarfile.open(tar_name_1, "w:gz") as tar:
        tar.add(src+'/r', arcname=os.path.basename(src+'/r'))
        print('Added: '+src+'\r'+' to '+tar_name_1)
        tar.add(src+'/b', arcname=os.path.basename(src+'/b'))
        print('Added: '+src+'\b'+' to '+tar_name_1)
        tar.add(src+'/g', arcname=os.path.basename(src+'/g'))
        print('Added: '+src+'\g'+' to '+tar_name_1)
        tar.add(src+'/fr', arcname=os.path.basename(src+'/fr'))
        print('Added: '+src+'\fr'+' to '+tar_name_1)
        tar.add(src+'/rgb_tif', arcname=os.path.basename(src+'/rgb_tif'))
        print('Added: '+src+'\rgb_tif'+' to '+tar_name_1)
        tar.add(src+'/frgb_tif', arcname=os.path.basename(src+'/frgb_tif'))
        print('Added: '+src+'\frgb_tif'+' to '+tar_name_1)
    with tarfile.open(tar_name_2, "w:gz") as tar:
        tar.add(src+'/rgb', arcname=os.path.basename(src+'/rgb'))
        print('Added: '+src+'\rgb_png'+' to '+tar_name_2)
    with tarfile.open(tar_name_3, "w:gz") as tar:
        tar.add(src+'/frgb', arcname=os.path.basename(src+'/frgb'))
        print('Added: '+src+'\frgb'+' to '+tar_name_3)

In [ ]:
#if imgs are 8bit: bitdepth = np.uint8; if imgs are 16bit: bitdepth = np.uint16
bitdepth = np.uint16

src = r'I:\Einar\Screen_1\trial'
src_mask = r'I:\Einar\Screen_1\trial\mask'

chan_1_int_upperQ=0.999
chan_2_int_upperQ=0.999
chan_3_int_upperQ=0.999
chan_4_int_upperQ=0.999

chan_1_upperQnr2=0.95
chan_2_upperQnr2=0.99
chan_3_upperQnr2=0.99
chan_4_upperQnr2=0.95

obj = 'cells'
height = 448
width = 448
#cell_size_cutoff = 2000

parent_obj = 'cells'
child_obj = 'nuclei'
second_child_obj = 'toxoplasma'
height =  448
width = 448
parent_size_cutoff = 2000
child_size_cutoff = 1000
parent_vs_child_cutoff = 0.5
chan = 'rgb'

In [ ]:
gray_to_color(src, chna_1, chan_2, chna_3, chna_4, chna_5, chna_6, chna_7, chna_8, chna_9, 
                  chan_1_int_upperQ, chan_2_int_upperQ, chan_3_int_upperQ, chan_4_int_upperQ, chan_5_int_upperQ, 
                  chan_6_int_upperQ, chan_7_int_upperQ, chan_8_int_upperQ, chan_9_int_upperQ:

In [ ]:
gray_to_color(src, chan_1_int_upperQ, chan_2_int_upperQ, chan_3_int_upperQ, chan_4_int_upperQ)

In [ ]:
show_stack(stack)

In [ ]:
rescal_intensity(src, chan_1_upperQnr2, chan_2_upperQnr2, chan_3_upperQnr2, chan_4_upperQnr2)

In [ ]:
#cellpose is in seperate notebook, run cellpose before this
#src = r'I:\Einar\Screen_1\trial'
mode = 'rgb'
imgs=[] #store all images
show_imgs(src, mode)

In [ ]:
create_test_folder(src, mode)

In [ ]:
#Select paramiters for cellpose segmentation

# Primary object segmentation, if nuclei are present set this object to nuclei
obj1 = 'nuclei' # Set an appropreate name
obj1_channel = 3 # Set the channel the object will be segmented from 
dim1 = 50 # Set diamiter in pixels (length in pixels)
mt1 = -2 # Threshold for cellpose, lower if objects are not detected, increase if to many objects are detected
gausian1 = 0

# Secondary object detection
obj2 = 'cells' # Set an appropreate name or set off
obj2_channel = 1 # Set the channel the object will be segmented from 
dim2 = 0 # Set diamiter in pixels (length in pixels)
mt2 = -1 # Threshold for cellpose, lower if object0s are not detected, increase if to many objects are detected
obj3_model = 'cyto2'
gausian2 = 2

#Tertery object detection
obj3 = 'toxoplasma' # Set an appropreate name or set off
obj3_channel = 2 # Set the channel the object will be segmented from 
dim3 = 30 # Set diamiter in pixels (length in pixels)
mt3 = 0 # Threshold for cellpose, lower if objects are not detected, increase if to many objects are detected
obj3_model = 'cyto' #set to yes if the tertery object is nucleated and you have a nuclear channel
mode = 'rgb' # Set to rgb or grey
gausian3 = 0

#Common paramiters
batch_size = 24 #Default is 8, higher values use more GPU VRAM
flow_threshold = 10
print(src)

if os.path.basename(src) != 'test':
    src = src+'/test'
    
print(src)

run_cellpose_3_objects(src, mode, 
                           obj1, obj1_channel, dim1, mt1,
                           obj2, obj2_channel, dim2, mt2,
                           obj3, obj3_channel, dim3, mt3, 
                           obj3_model, batch_size)

In [ ]:
#if imgs are 8bit: bitdepth = np.uint8; if imgs are 16bit: bitdepth = np.uint16
bitdepth = np.uint16

src = r'I:\Einar\Screen_1\trial'
src_mask = r'I:\Einar\Screen_1\trial\mask'

far_red_int_upperQ=0.999
red_int_upperQ=0.999
green_int_upperQ=0.999
blue_int_upperQ=0.999

far_red_upperQnr2=0.95
red_upperQnr2=0.99
green_upperQnr2=0.99
blue_upperQnr2=0.95

obj = 'cells'
height = 448
width = 448
#cell_size_cutoff = 2000

parent_obj = 'cells'
child_obj = 'nuclei'
second_child_obj = 'toxoplasma'
height =  448
width = 448
parent_size_cutoff = 2000
child_size_cutoff = 1000
parent_vs_child_cutoff = 0.5
chan = 'rgb'

In [ ]:
move_to_chan_folder(src)

In [ ]:
gray_to_color(src, far_red_int_upperQ, red_int_upperQ, green_int_upperQ, blue_int_upperQ)

In [ ]:
rescal_intensity(src, far_red_upperQnr2, red_upperQnr2, green_upperQnr2, blue_upperQnr2)

In [ ]:
#cellpose is in seperate notebook, run cellpose before this
src = r'C:\Users\einar\Desktop\omnipose_test'
mode = 'rgb'
imgs=[] #store all images
show_imgs(src, mode)

In [ ]:
create_test_folder(src, mode)

In [ ]:
#Select paramiters for cellpose segmentation

# Primary object segmentation, if nuclei are present set this object to nuclei
obj1 = 'nuclei' # Set an appropreate name
obj1_channel = 3 # Set the channel the object will be segmented from 
dim1 = 50 # Set diamiter in pixels (length in pixels)
mt1 = -2 # Threshold for cellpose, lower if objects are not detected, increase if to many objects are detected
gausian1 = 0

# Secondary object detection
obj2 = 'cells' # Set an appropreate name or set off
obj2_channel = 1 # Set the channel the object will be segmented from 
dim2 = 0 # Set diamiter in pixels (length in pixels)
mt2 = -1 # Threshold for cellpose, lower if object0s are not detected, increase if to many objects are detected
obj3_model = 'cyto2'
gausian2 = 2

#Tertery object detection
obj3 = 'toxoplasma' # Set an appropreate name or set off
obj3_channel = 2 # Set the channel the object will be segmented from 
dim3 = 30 # Set diamiter in pixels (length in pixels)
mt3 = 0 # Threshold for cellpose, lower if objects are not detected, increase if to many objects are detected
obj3_model = 'cyto' #set to yes if the tertery object is nucleated and you have a nuclear channel
mode = 'rgb' # Set to rgb or grey
gausian3 = 0

#Common paramiters
batch_size = 24 #Default is 8, higher values use more GPU VRAM
flow_threshold = 10
print(src)

In [ ]:
if os.path.basename(src) != 'test':
    src = src+'/test'
    
print(src)

In [ ]:
run_cellpose_3_objects(src, mode, 
                           obj1, obj1_channel, dim1, mt1,
                           obj2, obj2_channel, dim2, mt2,
                           obj3, obj3_channel, dim3, mt3, 
                           obj3_model, batch_size)

In [ ]:
src = r'C:\Users\einar\Desktop\omi'
mode = 'stack'
plot_array(src, mode)

In [ ]:
bitdepth = np.uint16
mode = 'stack'
nr_of_channels = 4
gray_to_color(src, nr_of_channels, mode)

In [ ]:
rescal_intensity(src, nr_of_channels, mode)

In [ ]:
gausian = 10
dim = 200
kernel = np.ones((gausian, gausian),np.float32)/(gausian*gausian)
image = cv2.filter2D(image,-1, kernel)
image_show = img_as_ubyte(image)
min = (dim*dim)/2

In [ ]:
file = r'C:\Users\einar\Desktop\omi\PRUWT_LHVS-04_TexRe.tif'
array = np.load(file)
chan_1, chan_2, chan_3, chan_4 = np.split(array, 4, axis=2)
image = cv2.merge((chan_1, chan_2, chan_3), 3) 
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # img for cellpose
image_show = img_as_ubyte(image) # img for matplotlib
image2 = image
print(image.shape)
model = models.Cellpose(gpu=True, model_type="cyto2")

In [ ]:
file = r'C:\Users\einar\Desktop\omi\PRUWT_LHVS-04_TexRe.tif'
image = cv2.imread(file,-1)
model = models.Cellpose(gpu=True, model_type="cyto2")

In [ ]:
masks, flows, styles, diams = model.eval(image,
                                          batch_size = 8, # (int (optional, default 8)) – number of 224x224 patches to run simultaneously on the GPU (can make smaller or bigger depending on GPU memory usage)
                                          channels = [1,3], # (list (optional, default None)) – list of channels, either of length 2 or of length number of images by 2. First element of list is the channel to segment (0=grayscale, 1=red, 2=green, 3=blue). Second element of list is the optional nuclear channel (0=none, 1=red, 2=green, 3=blue). For instance, to segment grayscale images, input [0,0]. To segment images with cells in green and nuclei in blue, input [2,3]. To segment one grayscale image and one image with cells in green and nuclei in blue, input [[0,0], [2,3]].
                                          channel_axis = None, # (int (optional, default None)) – if None, channels dimension is attempted to be automatically determined
                                          z_axis = None, # (int (optional, default None)) – if None, z dimension is attempted to be automatically determined
                                          normalize = True, # (bool (default, True)) – normalize data so 0.0=1st percentile and 1.0=99th percentile of image intensities in each channel
                                          invert = False, # (bool (optional, default False)) – invert image pixel intensity before running network
                                          rescale = True, # (float (optional, default None)) – resize factor for each image, if None, set to 1.0
                                          diameter = 200, # (float (optional, default None)) – diameter for each image (only used if rescale is None), if diameter is None, set to diam_mean
                                          do_3D = False, # (bool (optional, default False)) – set to True to run 3D segmentation on 4D image input
                                          anisotropy = None, # (float (optional, default None)) – for 3D segmentation, optional rescaling factor (e.g. set to 2.0 if Z is sampled half as dense as X or Y)
                                          net_avg = True, # (bool (optional, default True)) – runs the 4 built-in networks and averages them if True, runs one network if False
                                          augment = True, # (bool (optional, default False)) – tiles image with overlapping tiles and flips overlapped regions to augment
                                          tile = True, # (bool (optional, default True)) – tiles image to ensure GPU/CPU memory usage limited (recommended)
                                          tile_overlap = 0.1, # (float (optional, default 0.1)) – fraction of overlap of tiles when computing flows
                                          resample = True, # (bool (optional, default False)) – run dynamics at original image size (will be slower but create more accurate boundaries)
                                          interp = True, # (bool (optional, default True)) – interpolate during 2D dynamics (not available in 3D) (in previous versions it was False)
                                          flow_threshold = 10, # (float (optional, default 0.4)) – flow error threshold (all cells with errors below threshold are kept) (not used for 3D)
                                          mask_threshold = 0, # (float (optional, default 0.0)) – all pixels with value above threshold kept for masks, decrease to find more and larger masks
                                          # DEPRECEATED: dist_threshold = None, # (float (optional, default None) DEPRECATED) – use mask_threshold instead
                                          # DEPRECEATED: cellprob_threshold = secondary_ct, # (float (optional, default None) DEPRECATED) – use mask_threshold instead
                                          #compute_masks = True, # (bool (optional, default True)) – Whether or not to compute dynamics and return masks. This is set to False when retrieving the styles for the size model.
                                          min_size = 50, # (int (optional, default 15)) – minimum number of pixels per mask, can turn off with -1
                                          stitch_threshold = 0.0, # (float (optional, default 0.0)) – if stitch_threshold>0.0 and not do_3D, masks are stitched in 3D to return volume segmentation
                                          progress = True, # (pyqt progress bar (optional, default None)) – to return progress bar status to GUI
                                          omni = True, # (bool (optional, default False)) – use omnipose mask recontruction features
                                          #calc_trace = False, # (bool (optional, default False)) – calculate pixel traces and return as part of the flow
                                          verbose = True, # (bool (optional, default False)) – turn on additional output to logs for debugging
                                          transparency = False) #(bool (optional, default False)) – modulate flow opacity by magnitude instead of brightness (can use flows on any color background)

maski = masks
flowi = flows[0]
fig = plt.figure(figsize=(100,100))
plot.show_segmentation(fig, image_show, maski, flowi, channels=[1,3])
plt.tight_layout()
plt.show() #plt.subplot(122),plt.imshow(dst),plt.title('Averaging')